In [1]:
import os
import io
import subprocess
import glob
import re
from datetime import datetime
import pandas as pd
from scipy import stats
import numpy as np
import math
from pathlib import Path
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.gridspec as gs
import matplotlib.colors as mcol
from matplotlib.ticker import LinearLocator
from scipy.stats import gamma
from datetime import date
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, FileUpload, Button, Layout, IntProgress, Checkbox, Button, HBox, VBox, Output, Box, Label
from IPython.display import display, clear_output, Javascript, Image

import warnings
warnings.filterwarnings("ignore", category=UserWarning)

#calls the CentralAge.py file (should be in the same folder as project)
from CentralAge import *

In [2]:
%matplotlib inline

In [3]:
RAp_w = widgets.Select(
    options=[("7.5e-4 (FCT)",7.5E-4), ("7.17e-4 (DUR)",7.17E-4), ("5.3e-4 (ZIR)",5.3E-4), ("5.3e-4 (MONZ)",5.3E-4)],
    value=7.17E-4,
    rows=4,
    description='(R) Track Init:',
    disabled=False)

qAp_w = widgets.FloatText(
    value=0.93,
    description='(q) Etch Fact:',
    step=0.01,
    disabled=False,)

dAp_w = widgets.Select(
    options=[("3.21 (Ap)",3.21), ("4.7 (Zr)",4.7),("5.15 (Mz)",5.15)],
    value=3.21,
    rows=3,
    description='(d) Density:',
    disabled=False)

M238U_w = widgets.FloatText(
    value=238.051,
    description='M238U:',
    step=0.01,
    disabled=True)

g_w = widgets.FloatText(
    value=1.0,
    description='g:',
    step=0.01,
    disabled=False)

ld_w = widgets.FloatText(
    value=1.55125E-10,
    description='λd:',
    step=0.01,
    disabled=True)

lf_w = widgets.FloatText(
    value=8.52E-17,
    description='λf:',
    step=0.01,
    disabled=True)

No_w = widgets.FloatText(
    value=6.0221409E+23,
    description='No:',
    step=0.01,
    disabled=True)

def Xi_f(RAp_w, qAp_w, dAp_w, M238U_w, g_w, ld_w, lf_w, No_w):
    global Xi
    global RAp
    global qAp
    global dAp
    global M238U
    global g
    global ld
    global lf
    global No
    Xi = M238U_w/(lf_w*No_w*dAp_w*RAp_w*qAp_w)
    RAp = RAp_w
    qAp = qAp_w
    dAp = dAp_w
    M238U = M238U_w
    g = g_w
    ld = ld_w
    lf = lf_w
    No = No_w
    print ('Aggregate factor ξ:'" {:.4e}  t cm^2".format(Xi))
    print ('Equivalent to Vermeesch (2017) & IsoplotR ζ:'" {:.1f}  t μm^2".format(Xi*2*1000000))

def reset_values1(b):
    """Reset to inital values."""
    RAp_w.value = 7.17E-4
    qAp_w.value = 0.93
    dAp_w.value = 3.21

reset_button1 = widgets.Button(description = "Default Values")
reset_button1.on_click(reset_values1)

print('1) DEFINE VARIABLES AND CHECK CONSTANTS')
out1 = widgets.interactive_output(Xi_f, {'RAp_w': RAp_w, 'qAp_w': qAp_w, 'dAp_w': dAp_w, 'M238U_w': M238U_w,
                                     'g_w': g_w,'ld_w': ld_w, 'lf_w': lf_w, 'No_w': No_w})
display(widgets.HBox([widgets.VBox([RAp_w,qAp_w,dAp_w,out1]),
                      widgets.VBox([M238U_w,g_w,ld_w,lf_w,No_w])]),
       widgets.HBox([reset_button1]))

1) DEFINE VARIABLES AND CHECK CONSTANTS


In [4]:
sample_name_w = widgets.Text(
    #value='Sample_01',
    placeholder='Sample_01',
    description='Sample ID**:',
    disabled=False)

igsn_w = widgets.Text(
    #value='123456',
    placeholder='IGSN0011',
    description='IGSN**:',
    disabled=False)

analyst_w = widgets.Text(
    value='Your Name',
    placeholder='Your Name',
    description='Analyst**:',
    disabled=False)

collector_w = widgets.Text(
    placeholder='Your Name?',
    description='Collector:',
    disabled=False)

rock_type_w = widgets.Text(
    #value='Hello World',
    placeholder='Granite',
    description='Rock Type:',
    disabled=False)

country_w = widgets.Text(
    placeholder='Australia',
    description='Country:',
    disabled=False)

region_w = widgets.Textarea(
    #value='Hello World',
    placeholder='Snowy Mountains',
    description='Region:',
    disabled=False)

latitude_w = widgets.FloatText(
    value=-12.3456,
    #placeholder=-12.3456,
    step=None,
    description='Lat. (wgs84):',
    disabled=False)

longitude_w = widgets.FloatText(
    value=12.3456,
    #placeholder=12.3456,
    step=None,
    description='Lon. (wgs84):',
    disabled=False)

elevation_w = widgets.IntText(
    #value=np.nan,
    placeholder=100,
    description='elevation (m):',
    disabled=False)

    #These will likely remain constant

mineral_w = widgets.Select(
    options=[("Apatite","Apatite"), ("Zircon","Zircon"), ("Monazite","Monazite")],
    value='Apatite',
    rows=3,
    #placeholder='Apatite',
    description='Mineral:',
    disabled=False)

Ustandard_w = widgets.Text(
    value='Nist612',
    placeholder='Standard',
    description='Pimary Std:',
    disabled=False)

Intstandard_w = widgets.Select(
    options=[("Durango",'Durango'),
             ("NIST612",'Nist612'),
             ("NIST614",'Nist614'),
            ("MudTank",'MudTank'),
            ("Other",'other')],
    value='Durango',
    rows=5,
    description='2nd Std:',
    disabled=False)

spot_size_w = widgets.Text(
    value='30 µm',
    placeholder='30 µm',
    description='Spot Size:',
    disabled=False)

lab_name_w = widgets.Textarea(
    value='University of Melbourne Thermochronology',
    placeholder='University of ..',
    description='Lab Name:',
    disabled=False)

etchant_w = widgets.Text(
    value='5M HNO3',
    placeholder='5M HNO3',
    description='Etchant:',
    disabled=False)

etchant_w = widgets.Select(
    options=[("5M HNO3",'5M HNO3'), ("5.5M HNO3",'5.5M HNO3'),("6M HCl",'6M HCl')],
    value='5M HNO3',
    rows=3,
    description='Etchant:',
    disabled=False)

etching_time_w = widgets.Text(
    value='20s',
    placeholder='20s',
    description='Etch Time:',
    disabled=False)

etching_temp_w = widgets.Text(
    value='20C',
    placeholder='20C',
    description='Etch Temp.:',
    disabled=False)

rmr0pref_w = widgets.Select(
    options=[("Ketcham 2007",'2007'), ("Ketcham/Carlson 1999",'1999')],
    value='2007',
    rows=2,
    description='rmr0 model:',
    disabled=False)

eCLpref_w = widgets.Select(
    options=[("Ketcham 2007",'2007'), ("Ketcham 1999",'1999')],
    value='2007',
    rows=2,
    description='eCl model:',
    disabled=False)

def meta_f(sample_name_w, igsn_w, analyst_w, collector_w, rock_type_w,
           country_w, region_w, latitude_w, longitude_w, elevation_w,
          mineral_w, Ustandard_w, Intstandard_w, spot_size_w, lab_name_w, etchant_w,
          etching_time_w, etching_temp_w, rmr0pref_w, eCLpref_w):
    global sample_name
    global igsn
    global analyst
    global collector
    global rock_type
    global country
    global region
    global latitude
    global longitude
    global elevation
    global mineral
    global Ustandard
    global Intstandard
    global spot_size
    global lab_name
    global etchant
    global etching_time
    global etching_temp
    global rmr0pref
    global eCLpref
    sample_name = sample_name_w
    igsn = igsn_w
    analyst = analyst_w
    collector = collector_w
    rock_type = rock_type_w
    country = country_w
    region = region_w
    latitude = round(latitude_w,6)
    longitude = round(longitude_w,6)
    elevation = elevation_w
    mineral = mineral_w
    Ustandard = Ustandard_w
    Intstandard = Intstandard_w
    spot_size = spot_size_w
    lab_name = lab_name_w
    etchant = etchant_w
    etching_time = etching_time_w
    etching_temp = etching_temp_w
    rmr0pref = rmr0pref_w
    eCLpref = eCLpref_w
    

def reset_values2(b):
    """Clear to defined values."""
    sample_name_w.value = ''
    igsn_w.value = ''
    analyst_w.value = ''
    collector_w.value = ''
    rock_type_w.value = ''
    country_w.value = ''
    region_w.value = ''
    latitude_w.value = np.nan
    longitude_w.value = np.nan
    elevation_w.value = 0
    lab_name_w.value = ''

reset_button2 = widgets.Button(description = "Clear Inputs")
reset_button2.style.button_color = 'orange'
reset_button2.on_click(reset_values2)

def reset_values3(b):
    """Reset to defined values."""
    mineral_w.value = "Apatite"
    Ustandard_w.value = 'Nist612'
    Intstandard_w.value = 'Durango'
    spot_size_w.value = '30 µm'
    lab_name_w.value = 'University of Melbourne Thermochronology'
    etchant_w.value = '5M HNO3'
    etching_time_w.value = '20s'
    etching_temp_w.value = '20C'
    sample_name_w.value = ''
    analyst_w.value = 'Your Name'
    collector_w.value = ''
    rock_type_w.value = ''
    country_w.value = ''
    region_w.value = ''
    latitude_w.value = -12.3456
    longitude_w.value = 12.3456
    elevation_w.value = 0

reset_button3 = widgets.Button(description = "Default Values")
#reset_button3.style.button_color = 'gray'
reset_button3.on_click(reset_values3)

print("2) OPTIONAL META DATA USED IN EXPORT TABLES")
print("** If left blank, will attempt to retrieve information from FastTracks file")
out2 = widgets.interactive_output(meta_f, {'sample_name_w': sample_name_w,'igsn_w': igsn_w, 'analyst_w': analyst_w, 'collector_w': collector_w,
                                           'rock_type_w': rock_type_w,'country_w': country_w,'region_w': region_w,
                                           'latitude_w': latitude_w, 'longitude_w': longitude_w, 'elevation_w': elevation_w,
                                          'mineral_w': mineral_w, 'Ustandard_w': Ustandard_w, 'Intstandard_w': Intstandard_w,
                                           'spot_size_w': spot_size_w, 'lab_name_w': lab_name_w, 'etchant_w': etchant_w,
                                           'etching_time_w': etching_time_w,'etching_temp_w': etching_temp_w,
                                          'rmr0pref_w': rmr0pref_w,'eCLpref_w': eCLpref_w})

display(widgets.HBox([widgets.VBox([sample_name_w,igsn_w,analyst_w, collector_w, rock_type_w,
           country_w, region_w, latitude_w, longitude_w, elevation_w, Ustandard_w, Intstandard_w,]),
                      
              widgets.VBox([mineral_w, spot_size_w, lab_name_w, etchant_w,
          etching_time_w, etching_temp_w, rmr0pref_w, eCLpref_w])]),
        
        widgets.HBox([reset_button3,reset_button2]))


2) OPTIONAL META DATA USED IN EXPORT TABLES
** If left blank, will attempt to retrieve information from FastTracks file


In [5]:
print('3) INPUT DIRECTORY TO PATH WHERE FILES WILL BE SAVED')
files_save_to_w = widgets.Text(
    value='/Users/USER/FOLDER/SAVE_HERE',
    placeholder='/Users/USER/Desktop/AFT_Calc',
    description='Save To:',
    layout=Layout(width='80%', height='100%'),
    disabled=False)

def clear_folder(b):
    files_save_to_w.value = ""
    out_p.clear_output()
clear_button = widgets.Button(description = "Clear Path")
clear_button.style.button_color = 'orange'
clear_button.on_click(clear_folder)


display(widgets.VBox([widgets.HBox([files_save_to_w]),widgets.HBox([clear_button])]))

3) INPUT DIRECTORY TO PATH WHERE FILES WILL BE SAVED


In [6]:
counts_paths_test = []
ICPMS_paths_test = []
lengths_paths_test = []
counts_paths = []
ICPMS_paths = []
lengths_paths = []


files_get_from_w = widgets.Text(
    value='/Users/USER/FOLDER/DATA_FOLDER',
    placeholder='/Users/USER/FOLDER/DATA_FOLDER Counts.csv & Lengths.csv & ICPMS.txt',
    #description='Look In:',
    layout=Layout(width='70%', height='100%'),
    disabled=False)

countskey_w = widgets.Text(
    value='Counts',
    placeholder='Counts',
    layout=Layout(width='20%', height='100%'),
    disabled=False)

lengthskey_w = widgets.Text(
    value='Lengths',
    placeholder='Lengths',
    layout=Layout(width='20%', height='100%'),
    disabled=False)

icpmskey_w = widgets.Text(
    value='ICPMS',
    placeholder='ICPMS',
    layout=Layout(width='20%', height='100%'),
    disabled=False)

def test_retrieve(b_test): 
    global counts_paths_test
    counts_paths_test = []
    for root, dirs, files in os.walk(files_get_from_w.value):
        for f in files:
            fullpath = os.path.join(root,f)
            if countskey_w.value in fullpath and sample_name in fullpath and ".csv" in fullpath:
                counts_paths_test.append(fullpath)
                if len(counts_paths_test) >2:
                    break
    with out_t:
        if len(counts_paths_test) == 0:
            print("!!! Couldn't Find Minimum Data Files Using PATH")
        elif len(counts_paths_test) >2:
            print("!!! Found multiple non-unique options using sample ID, check ID field (STEP 2)")
        else:
            print ("Track Counts File: {}".format(counts_paths_test[0].split("/")[-1]))
    if counts_paths_test ==[]:
        with out_t:
            print('Searching in path.. {0}\nNo File Matching Counts/{1}.csv found in path'.format(files_get_from_w.value,sample_name))
            
    global ICPMS_paths_test
    ICPMS_paths_test = []
    for root, dirs, files in os.walk(files_get_from_w.value):
        for f in files:
            fullpath = os.path.join(root,f)
            if icpmskey_w.value in fullpath and sample_name in fullpath and ".txt" in fullpath:
                ICPMS_paths_test.append(fullpath)
                if len(counts_paths_test) >1:
                    break
                else:
                    with out_t:
                        print ("ICPMS File: {}".format(ICPMS_paths_test[0].split("/")[-1]))
    if ICPMS_paths_test ==[]:
        with out_t:
            print('No File Matching ICPMS/{0}.txt found in path'.format(sample_name))
    
                
    global lengths_paths_test
    lengths_paths_test = []
    for root, dirs, files in os.walk(files_get_from_w.value):
        for f in files:
            fullpath = os.path.join(root,f)
            if lengthskey_w.value in fullpath and sample_name in fullpath and ".csv" in fullpath:
                lengths_paths_test.append(fullpath)
                if len(counts_paths_test) >1:
                    break
                else:
                    with out_t:
                        print ("Lengths File: {}".format(lengths_paths_test[0].split("/")[-1]))
    if lengths_paths_test ==[]:
        with out_t:
            print('No File Matching Lengths/{0}.csv found in path'.format(sample_name))

    if ICPMS_paths_test and counts_paths_test != [] and len(counts_paths_test) ==1:
        with out_t:
            print ('>>Required Files Found, if correct "Store Files" and continue')
            
def retrieve(b_ret): 
    global counts_paths
    global ICPMS_paths
    global lengths_paths
    if counts_paths_test != []:
        counts_paths = counts_paths_test[0]
    else:
        counts_paths = []
    if ICPMS_paths_test != []:
        ICPMS_paths = ICPMS_paths_test[0]
    else:
        ICPMS_paths = []
    if lengths_paths_test != []:
        lengths_paths = lengths_paths_test[0]
    else:
        lengths_paths = []
        
    with out_r:
        if counts_paths != []:
            print ("Track Counts File (stored): {}".format(counts_paths.split("/")[-1]))
        else:
            print ("No Counts File Found, Try testing the path first.\n   .csv files should be in PATH/Counts/Sample_01.csv")
    with out_r:
        if ICPMS_paths != []:
            print ("ICPMS File (stored): {}".format(ICPMS_paths.split("/")[-1]))
        else:
            print ("No ICPMS File Found, Try testing the path first\n    .txt files should be in PATH/ICPMS/Sample_01.csv")
    with out_r:
        if lengths_paths != []:
            print ("Lengths File (stored): {}".format(lengths_paths.split("/")[-1]))
        else:
            print ("No Lengths File Found, is that correct?")
    with out_r:
        if ICPMS_paths_test and counts_paths_test != []:
            print ('>>Looks Good! Commit and Continue')
        
test_ret_button = widgets.Button(description = "Test Folder Path", button_style='primary')
test_ret_button.on_click(test_retrieve)

ret_button = widgets.Button(description = "Store Found Files", button_style='primary')
ret_button.on_click(retrieve)

                    
out_t = Output()
out_r = Output()

def clear_retrieve(b):
    files_get_from_w.value = ""
clear_button2 = widgets.Button(description = "Clear Path", layout=Layout(width='20%', height='100%'))
clear_button2.style.button_color = 'orange'
clear_button2.on_click(clear_retrieve)


searchitems_1 = [Label('Path to Parent Folder w/ Data Folder(s):'),files_get_from_w]
searchitems_2 = [clear_button2]
searchitems_3 = [Label('Folder or Data File Names Contain (case-sensitive):'),countskey_w, lengthskey_w,icpmskey_w]


search_layout = Layout(display='flex',
                    flex_flow='row',
                    align_items='stretch',
                    width='90%',
                    height='100%')

search_layout2 = Layout(display='flex-grow',
                    flex_flow='row',
                    align_items='center',
                    width='80%')

search_layout3 = Layout(display='flex',
                    flex_flow='row',
                    align_items='stretch',
                    width='90%')

searchbox_1 = Box(children=searchitems_1, layout=search_layout)
searchbox_2 = Box(children=searchitems_2, layout=search_layout2)
searchbox_3 = Box(children=searchitems_3, layout=search_layout3)



        

print ('4) AUTO-RETRIEVE DATA FILES LOCATED IN PARENT/SUB FOLDER(s) by SAMPLED ID -OR- UPLOAD')
display(VBox([searchbox_1,searchbox_2,searchbox_3]))

print("REQUIRED>> (if auto-retrieving, make sure to ''store'')")
display(widgets.HBox([test_ret_button,ret_button]))
display(out_t,out_r)


4) AUTO-RETRIEVE DATA FILES LOCATED IN PARENT/SUB FOLDER(s) by SAMPLED ID -OR- UPLOAD


REQUIRED>> (if auto-retrieving, make sure to ''store'')


Output()

Output()

In [31]:
counts_file_w = FileUpload(accept='.csv',
                      description='Counts (csv)*',
                      multiple=False)
icpms_file_w = FileUpload(accept='.txt',
                      description='ICPMS (txt)*',
                      multiple=False)
lengths_file_w = FileUpload(accept='.csv',
                      description='Lengths (csv)',
                      multiple=False)

def upload_c(counts_file_w):
    if counts_file_w != {}:
        print (next(iter(counts_file_w)))
def upload_i(icpms_file_w):
    if icpms_file_w != {}:
        print (next(iter(icpms_file_w)))
def upload_l(lengths_file_w):
    if lengths_file_w != {}:
        print (next(iter(lengths_file_w)))

out_counts = widgets.interactive_output(upload_c, {'counts_file_w': counts_file_w})  
out_icpms = widgets.interactive_output(upload_i, {'icpms_file_w': icpms_file_w}) 
out_lengths = widgets.interactive_output(upload_l, {'lengths_file_w': lengths_file_w}) 
    

print("~~Option 2~~")
print("4.1) MANUALLY UPLOAD DATA FILES")
display(widgets.HBox([widgets.VBox([counts_file_w,out_counts]),
              widgets.VBox([icpms_file_w,out_icpms]),
             widgets.VBox([lengths_file_w,out_lengths])]))

~~Option 2~~
4.1) MANUALLY UPLOAD DATA FILES


In [32]:
def clear_files(b):
    out_t.clear_output()
    out_r.clear_output()
    global counts_paths_test
    global ICPMS_paths_test
    global lengths_paths_test
    global counts_paths
    global ICPMS_paths
    global lengths_paths
    counts_paths_test = []
    ICPMS_paths_test = []
    lengths_paths_test = []
    counts_paths = []
    ICPMS_paths = []
    lengths_paths = []
    out_counts.clear_output()
    out_icpms.clear_output()
    out_lengths.clear_output()
    display(Javascript('IPython.notebook.execute_cell()'))
    display(Javascript('IPython.notebook.execute_cells_below()'))
    
clear_button3 = widgets.Button(description = "Clear Saved Data Files (reset to begin new sample)",
                               layout=Layout(width='80%'), button_style='danger')
clear_button3.on_click(clear_files)


out_p = Output()
@out_p.capture(clear_output=True)
def store_files(b):
    global save_to
    if files_save_to_w.value == '':
        print("!! No place to save files, input working directory (STEP 3) !!")
    elif os.access(files_save_to_w.value, os.W_OK) == False:
        print("!! Cannot continue, path to save (STEP 3) is either incorrect format or read-only !!")
    else:
        save_to = '{0}/FTAge_Calc/{1}'.format(files_save_to_w.value,sample_name)
        display(Javascript('IPython.notebook.execute_cells([9,10,11,12])'))
        with out_p:
            print('Files save to:',save_to)
        if not os.path.exists(save_to): os.makedirs(save_to)  



store_button = widgets.Button(description = "Commit Files & Continue",
                              layout=Layout(width='80%'),button_style='success')
store_button.on_click(store_files)


print("5) READ THE UPLOADED FILES & CONTINUE  -or- RESTART")
display(widgets.VBox([store_button,clear_button3,out_p]))


5) READ THE UPLOADED FILES & CONTINUE  -or- RESTART


In [34]:
#necessry blank cell?

In [35]:
##Read Counts File##
if counts_file_w.value == {} and counts_paths == [] and icpms_file_w.value == {} and ICPMS_paths == []:
    print (">> No Files, (Hit ~Commit~ button above to refresh)")
    
else:
    
    if counts_file_w.value != {}:
        counts_content = counts_file_w.value[next(iter(counts_file_w.value))]['content']
        counts_data = pd.read_csv(io.BytesIO(counts_content), skiprows=4, error_bad_lines=False)
        if counts_data.iat[0,0] != "Grain01":
            counts_data = pd.read_csv(io.BytesIO(counts_content), skiprows=5, error_bad_lines=False)
            #AUTO-generate some meta-data from the counts file header
        cd_headeronly = pd.read_csv(io.BytesIO(counts_content), error_bad_lines=False, header=None)
        
    elif counts_file_w.value == {} and counts_paths != []:
        counts_data = pd.read_csv(counts_paths, skiprows=4, error_bad_lines=False)  
        if counts_data.iat[0,0] != "Grain01":
            counts_data = pd.read_csv(counts_paths, skiprows=5, error_bad_lines=False)
            #AUTO-generate some meta-data from the counts file header
        cd_headeronly = pd.read_csv(counts_paths, error_bad_lines=False, header=None)
    else:
        counts_data = None
        print('>>>>>No Counts File Found!\nThis is required to continue!\n_________________')
        
    if counts_data is not None:
        sample_FT = cd_headeronly[0][0] #Uses this value only if Sample Name left blank at start
        igsn_FT = cd_headeronly[1][2] #Uses this value only if IGSN left blank at start
        analyst_FT = cd_headeronly[1][3] #Uses this value only if Analyst left blank at start
        software = cd_headeronly[0][1]
        if cd_headeronly[0][2] == "Pixel Calibration X:":
            igsn_FT = '--'
        if cd_headeronly[0][3] == "Pixel Calibration Y:":
            analyst_FT = '--'
            #px and py no longer included in latest FastTracks
        #px = pd.to_numeric(cd_headeronly[1][2])
        #py = pd.to_numeric(cd_headeronly[1][3])
        date_meas = pd.to_datetime(cd_headeronly[2][0], infer_datetime_format=True)
        print(">>Counts file found, variables assigned :)\nSoftware: {0}\nDate Measured: {1}\n\nSample: {2}\nNo. Grains: {3}\n_________________".format(software,date_meas,sample_FT,counts_data['Grain/Mica'].count()))
        #counts_data.head()


        ##Read ICPMS File##
        if icpms_file_w.value != {}:
            icpms_content = icpms_file_w.value[next(iter(icpms_file_w.value))]['content']
                #Selects only the columns we need
            icpms_data = pd.read_csv(io.BytesIO(icpms_content),delimiter="\t",
                usecols=["Unnamed: 0", "U_ppm_m238", "U_ppm_m238_Int2SE", "Time",
                         "Ca43_CPS", "Ca43_CPS_Int2SE", "Th_ppm_m232", "Th_ppm_m232_Int2SE"],)
        elif icpms_file_w.value == {} and ICPMS_paths != []:
                #Selects only the columns we need
            icpms_data = pd.read_csv(ICPMS_paths,delimiter="\t",
                usecols=["Unnamed: 0", "U_ppm_m238", "U_ppm_m238_Int2SE", "Time",
                         "Ca43_CPS", "Ca43_CPS_Int2SE", "Th_ppm_m232", "Th_ppm_m232_Int2SE"],)
        else:
            icpms_data = None
            print('No ICPMS File Found!\nThis is required!\n_________________')

        if icpms_data is not None:
                #Renames the unnamed column at column 0
            icpms_data.rename( columns={'Unnamed: 0':'point_name'}, inplace=True )
                # need to convert certain columns to floats instead of string for plotting, also fill nan with 0
            cols_numeric = ["U_ppm_m238", "U_ppm_m238_Int2SE", "Ca43_CPS", "Ca43_CPS_Int2SE",
                            "Th_ppm_m232", "Th_ppm_m232_Int2SE"]
            icpms_data[cols_numeric] = icpms_data[cols_numeric].apply(pd.to_numeric, errors='coerce', axis=1).fillna(0)
            ####DEFINE THE STANDARDS USED BASED ON POINT NAMES####
            #Information for standards used, you might need to add more if you've used other ones
            std_dur = icpms_data[icpms_data['point_name'].str.contains('Dur', na=False, case=False)]
            std_612 = icpms_data[icpms_data['point_name'].str.contains('612', na=False, case=False)]
            std_614 = icpms_data[icpms_data['point_name'].str.contains('614', na=False, case=False)]
            std_mt = icpms_data[icpms_data['point_name'].str.contains('Mud', na=False, case=False)]
            output = icpms_data[icpms_data['point_name'].str.contains('Output', na=False, case=False)]
            #resets the index row number for each standards data frame
            std_dur.reset_index(drop=True, inplace=True)
            std_612.reset_index(drop=True, inplace=True)
            std_614.reset_index(drop=True, inplace=True)
            std_mt.reset_index(drop=True, inplace=True)
            output.reset_index(drop=True, inplace=True)
                #combines the counts data and icpms data at the correct location to align
            age_df = pd.concat([counts_data, output], axis=1)
                #replaces the word "Grain" before each grain to make it easier to handle later
            age_df['Grain/Mica'].replace(regex=True,inplace=True,to_replace='Grain',value='')
                #see the data header if you want> (remove .head() to see full data)
            #age_df
            print(">>ICPMS file found, variables assigned :)\nFirst Unknown: {0}\nNo. of Unknowns: {1}\n_________________".format(output['point_name'][0],output['point_name'].count()))


        ##Read Lengths File##
        if lengths_file_w.value != {}:
            lengths_content = lengths_file_w.value[next(iter(lengths_file_w.value))]['content']
            lengths_data = pd.read_csv(io.BytesIO(lengths_content), skiprows=4)
        elif lengths_file_w.value == {} and lengths_paths !=[]:
            lengths_data = pd.read_csv(lengths_paths, skiprows=4)   
        else:
            print(">>No lengths file found\n_________________")
            lengths_content = '--'
            lengths_data = '--'
            true_length = '--'
            mtl = '--'
            mtl_sd = '--'
            l_no = '--'
            mtl_var = '--'
            Dpar_lengths = '--'
            Dpar_lengths_mean = '--'
            rmr0D_lengths = '--'
            rmr0D_lengths_mean = '--'
            rmr0D_lengths_sdm = '--'

        if lengths_data is not '--':
            true_length = lengths_data["True Length"]
            mtl = round(true_length.mean(),2)
            mtl_sd = round(true_length.std(),2)
            l_no = true_length.count()
            mtl_var = round(mtl_sd/np.sqrt(l_no),2)
            Dpar_lengths = round(lengths_data["Average DPar(µmm)"],2)
            Dpar_lengths_mean = round(Dpar_lengths.mean(),2)
            if etchant == '5M HNO3'and rmr0pref == '2007':
                rmr0D_lengths = round(0.84*((4.58-(0.9231*Dpar_lengths+0.2515))/2.98)**(0.21),3)
            elif etchant == '5M HNO3'and rmr0pref == '1999':
                rmr0D_lengths = round(1-np.exp(0.647*((0.9231*Dpar_lengths+0.2515)-1.75)-1.834),3)
            elif etchant == '5.5M HNO3' and rmr0pref == '2007':
                rmr0D_lengths = round(0.84*((4.58-(Dpar_lengths))/2.98)**(0.21),3)
            elif etchant == '5.5M HNO3'and rmr0pref == '1999':
                rmr0D_lengths = round(1-np.exp(0.647*(Dpar_lengths-1.75)-1.834),3)
            rmr0D_lengths_mean = round(rmr0D_lengths.mean(),3)
            rmr0D_lengths_sdm = round(rmr0D_lengths.std(),3)
            print(">>Lengths file found, variables assigned :)\nMTL: {0:.2f}±{1:.1f}\nNo. Lengths: {2}".format(mtl,mtl_sd,l_no))


        if counts_file_w.value != {} or counts_paths != []:
            ####PLOT SOME FIGURES FROM THE ICPMS DATA####

            #define the accepted values for standards to use in plots
            dur_value=12.2
            dur_err=0.04
            mt_value=3
            mt_err=0.02
            nist612_value=37.38
            nist612_err=0.08
            nist614_value=0.823
            nist614_err=0.002
            if Intstandard == 'other':
                mean_intstd = '--'
                meanerr_intstd = '--'

            if len(std_dur['point_name']) != 0:
                #####DURNAGO
                #convert spot time to seconds
                df_time_dur = pd.to_datetime(std_dur["Time"])
                x_dur = ((df_time_dur.dt.hour*60+df_time_dur.dt.minute)*60 + df_time_dur.dt.second)*10**(-4)
                #find and define the Uppm
                y_dur = std_dur["U_ppm_m238"]
                y_edur = std_dur["U_ppm_m238_Int2SE"]/2
                #define the mean line of the Uppm±SD
                y_mdur = [np.mean(y_dur)]*len(x_dur)
                y_medur = [np.mean(y_edur)]*len(x_dur)
                
                if Intstandard == 'Durango':
                    mean_intstd =round(y_mdur[0], 1)
                    meanerr_intstd = round(y_medur[0], 1)

                
            if len(std_mt['point_name']) != 0:
                #####MUD_TANK
                #convert spot time to seconds
                df_time_mt = pd.to_datetime(std_mt["Time"])
                x_mt = ((df_time_mt.dt.hour*60+df_time_mt.dt.minute)*60 + df_time_mt.dt.second)*10**(-4)
                #find and define the Uppm
                y_mt = std_mt["U_ppm_m238"]
                y_emt = std_mt["U_ppm_m238_Int2SE"]/2
                #define the mean line of the Uppm±SD
                y_mmt = [np.mean(y_mt)]*len(x_mt)
                y_memt = [np.mean(y_emt)]*len(x_mt)
                
                if Intstandard == 'MudTank':
                    mean_intstd =round(y_mmt[0], 1)
                    meanerr_intstd = round(y_memt[0], 1)

            ######OUTPUT(SAMPLES)
            #convert spot time to seconds
            df_time_output = pd.to_datetime(output["Time"])
            x_output = ((df_time_output.dt.hour*60+df_time_output.dt.minute)*60 + df_time_output.dt.second)*10**(-4)
            #find and define the Uppm
            y_output = output["U_ppm_m238"]
            y_eoutput = output["U_ppm_m238_Int2SE"]
            #define the mean line of the Uppm±SD
            y_moutput = [np.mean(y_output)]*len(x_output)
            y_meoutput = [np.mean(y_eoutput)]*len(x_output)

            if len(std_612['point_name']) != 0:
                #####NIST612
                #convert spot time to seconds
                df_time_nist612 = pd.to_datetime(std_612["Time"])
                x_nist612 = ((df_time_nist612.dt.hour*60+df_time_nist612.dt.minute)*60 + df_time_nist612.dt.second)*10**(-4)
                #find and define the Uppm
                y_nist612 = std_612["U_ppm_m238"]
                y_enist612 = std_612["U_ppm_m238_Int2SE"]/2
                #define the mean line of the Uppm±SD
                y_mnist612 = [np.mean(y_nist612)]*len(x_nist612)
                y_menist612 = [np.mean(y_enist612)]*len(x_nist612)
                
                if Intstandard == 'Nist612':
                    mean_intstd =round(y_mnist612[0], 1)
                    meanerr_intstd = round(y_menist612[0], 1)

            if len(std_614['point_name']) != 0:
                    #####NIST614
                    #convert spot time to seconds
                df_time_nist614 = pd.to_datetime(std_614["Time"])
                x_nist614 = ((df_time_nist614.dt.hour*60+df_time_nist614.dt.minute)*60 + df_time_nist614.dt.second)*10**(-4)
                    #find and define the Uppm
                y_nist614 = std_614["U_ppm_m238"]
                y_enist614 = std_614["U_ppm_m238_Int2SE"]/2
                    #define the mean line of the Uppm±SD
                y_mnist614 = [np.mean(y_nist614)]*len(x_nist614)
                y_menist614 = [np.mean(y_enist614)]*len(x_nist614)
                
                if Intstandard == 'Nist614':
                    mean_intstd =round(y_mnist614[0], 1)
                    meanerr_intstd = round(y_menist614[0], 1)


            #####FIGURES######
            #setup the figure (fig size is in inches)
            fig, axs = plt.subplots(2, 2, figsize=(9,9))

            if len(std_614['point_name']) == 0:
                    ####PLOT NIST612 if NIST614 was not shot
                    #create a color span on plot representing the accepted value for the standard
                axs[0,1].axhspan(nist612_value-nist612_err, nist612_value+nist612_err, alpha=0.1, color='cornflowerblue')
                    #plot the measured mean Uppm line in ref material
                axs[0,1].plot(x_nist612,y_mnist612, label='Mean', linestyle='-', color='#8B0000', alpha=0.5)
                axs[0,1].text(np.mean(x_nist612), np.mean(y_mnist612) + np.mean(y_mnist612)*0.01,'mean: {0:.2f}'" "u'\xb1'" "'{1:.2f}'" "'ppm' .format(round(y_mnist612[0], 2),round(y_menist612[0], 2)),fontsize=11, ha='center', color='#8B0000', zorder=3)
                axs[0,1].plot(np.unique(x_nist612), np.poly1d(np.polyfit(x_nist612, y_nist612, 1))(np.unique(x_nist612)), linestyle='--', color='k', alpha=0.2)
                    #plot the scatter points with error bars
                axs[0,1].scatter(x_nist612,y_nist612, label='Data', marker='o', color='#8B0000', s=25, zorder=2, alpha=.5)
                axs[0,1].errorbar(x_nist612,y_nist612, yerr=y_enist612, linestyle="None", color='k', alpha=.3, zorder=1)
                    #title and x,y labels
                axs[0,1].set_title('Nist-612', fontsize=12)
            else:
                    ####Plot NIST614 if it was shot
                    #create a color span on plot representing the accepted value for the standard
                axs[0,1].axhspan(nist614_value-nist614_err, nist614_value+nist614_err, alpha=0.1, color='cornflowerblue')
                    #plot the measured mean Uppm line in ref material
                axs[0,1].plot(x_nist614,y_mnist614, label='Mean', linestyle='-', color='#8B0000', alpha=0.5)
                axs[0,1].text(np.mean(x_nist614), np.mean(y_mnist614) + np.mean(y_mnist614)*0.01,'mean: {0:.2f}'" "u'\xb1'" "'{1:.2f}'" "'ppm' .format(round(y_mnist614[0], 2),round(y_menist614[0], 2)),fontsize=11, ha='center', color='#8B0000', zorder=3)
                axs[0,1].plot(np.unique(x_nist614), np.poly1d(np.polyfit(x_nist614, y_nist614, 1))(np.unique(x_nist614)), linestyle='--', color='k', alpha=0.2)
                    #plot the scatter points with error bars
                axs[0,1].scatter(x_nist614,y_nist614, label='Data', marker='o', color='#8B0000', s=25, zorder=2, alpha=.5)
                axs[0,1].errorbar(x_nist614,y_nist614, yerr=y_enist614, linestyle="None", color='k', alpha=.3, zorder=1)
                    #title and x,y labels
                axs[0,1].set_title('Nist-614', fontsize=12)
                
            if len(std_dur['point_name']) != 0:
                ####DURANGO
                #create a color span on plot representing the accepted value for the standard
                axs[1,0].axhspan(dur_value-dur_err, dur_value+dur_err, alpha=0.1, color='cornflowerblue')
                #plot the measured mean Uppm line in ref material
                axs[1,0].plot(x_dur,y_mdur, label='Mean', linestyle='-', color='#8B0000', alpha=0.5)
                axs[1,0].text(np.mean(x_dur), np.mean(y_mdur) + np.mean(y_mdur)*0.01,'mean: {0:.2f}'" "u'\xb1'" "'{1:.2f}'" "'ppm' .format(round(y_mdur[0], 2),round(y_medur[0], 2)),fontsize=11, ha='center', color='#8B0000', zorder=3)
                axs[1,0].plot(np.unique(x_dur), np.poly1d(np.polyfit(x_dur, y_dur, 1))(np.unique(x_dur)), linestyle='--', color='k', alpha=0.2)
                #plot the scatter points with error bars
                axs[1,0].scatter(x_dur,y_dur, label='Data', marker='o', color='#8B0000', s=25, zorder=2, alpha=.5)
                axs[1,0].errorbar(x_dur,y_dur, yerr=y_edur, linestyle="None", color='k', alpha=.3, zorder=1)
                #title and x,y labels
                axs[1,0].set_title('Durango', fontsize=12)

            if len(std_mt['point_name']) != 0:
                ####MUD_TANK
                #create a color span on plot representing the accepted value for the standard
                axs[1,1].axhspan(mt_value-mt_err, mt_value+mt_err, alpha=0.1, color='cornflowerblue')
                #plot the measured mean Uppm line in ref material
                axs[1,1].plot(x_mt,y_mmt, label='Mean', linestyle='-', color='#8B0000', alpha=0.5)
                axs[1,1].text(np.mean(x_mt), np.mean(y_mmt) + np.mean(y_mmt)*0.015,'mean: {0:.2f}'" "u'\xb1'" "'{1:.2f}'" "'ppm' .format(round(y_mmt[0], 2),round(y_memt[0], 2)),fontsize=11, ha='center', color='#8B0000', zorder=3)
                axs[1,1].plot(np.unique(x_mt), np.poly1d(np.polyfit(x_mt, y_mt, 1))(np.unique(x_mt)), linestyle='--', color='k', alpha=0.2)
                #plot the scatter points with error bars
                axs[1,1].scatter(x_mt,y_mt, label='Data', marker='o', color='#8B0000', s=25, zorder=2, alpha=.5)
                axs[1,1].errorbar(x_mt,y_mt, yerr=y_emt, linestyle="None", color='k', alpha=.3, zorder=1)
                #title and x,y labels
                axs[1,1].set_title('Mud-Tank', fontsize=12)

            ####OUTPUT(SAMPLES)
            #plot the measured mean Uppm line in ref material
            axs[0,0].plot(x_output,y_moutput, label='Mean', linestyle='-', color='#8B0000', alpha=0.5)
            axs[0,0].text(np.mean(x_output), np.mean(y_moutput) + np.mean(y_moutput)*0.05,'mean: {0:.2f}'" "u'\xb1'" "'{1:.2f}'" "'ppm' .format(round(y_moutput[0], 2),round(y_meoutput[0], 2)),fontsize=11, ha='center', color='#8B0000', zorder=3)
            axs[0,0].plot(np.unique(x_output), np.poly1d(np.polyfit(x_output, y_output, 1))(np.unique(x_output)), linestyle='--', color='k', alpha=0.2)
            #plot the scatter points with error bars
            axs[0,0].scatter(x_output,y_output, label='Data', marker='o', color='#8B0000', s=25, zorder=2, alpha=.5)
            axs[0,0].errorbar(x_output,y_output, yerr=y_eoutput, linestyle="None", color='k', alpha=.3, zorder=1)
            axs[0,0].set_yscale('log')
            #title and x,y labels
            axs[0,0].set_title('Sample: {}' .format(sample_name), fontsize=12)

            for ax in axs.flat:
                ax.set(xlabel='time 10e-4 (s)', ylabel='[U] ppm')
            # Hide x labels and tick labels for top plots and y ticks for right plots.
            #for ax in axs.flat:
            #    ax.label_outer()

            #padding between sublpots
            fig.tight_layout(pad=1.5)

            #save the figure to file, location defined at the start
            plt.savefig("{0}/{1}_Standards.pdf".format(save_to, sample_name), bbox_inches='tight', transparent=True)




>>Counts file found, variables assigned :)
Software: FastTracks: v3.2.15
Date Measured: 2022-05-18 15:13:26

Sample: 17SB-46A
No. Grains: 35
_________________
>>ICPMS file found, variables assigned :)
First Unknown: Output_1_1
No. of Unknowns: 35
_________________


b'Skipping line 5: expected 3 fields, saw 7\nSkipping line 6: expected 3 fields, saw 7\nSkipping line 7: expected 3 fields, saw 7\nSkipping line 8: expected 3 fields, saw 7\nSkipping line 9: expected 3 fields, saw 7\nSkipping line 10: expected 3 fields, saw 7\nSkipping line 11: expected 3 fields, saw 7\nSkipping line 12: expected 3 fields, saw 7\nSkipping line 13: expected 3 fields, saw 7\nSkipping line 14: expected 3 fields, saw 7\nSkipping line 15: expected 3 fields, saw 7\nSkipping line 16: expected 3 fields, saw 7\nSkipping line 17: expected 3 fields, saw 7\nSkipping line 18: expected 3 fields, saw 7\nSkipping line 19: expected 3 fields, saw 7\nSkipping line 20: expected 3 fields, saw 7\nSkipping line 21: expected 3 fields, saw 7\nSkipping line 22: expected 3 fields, saw 7\nSkipping line 23: expected 3 fields, saw 7\nSkipping line 24: expected 3 fields, saw 7\nSkipping line 25: expected 3 fields, saw 7\nSkipping line 26: expected 3 fields, saw 7\nSkipping line 27: expected 3 fields

KeyError: 'True Length'

In [ ]:
################################IMPORTANT CELL#####################################
if counts_file_w.value != {} or counts_paths != []:
    remove_grains_w = [widgets.ToggleButton(True,description="Grain {0}".format(g),
                                        button_style='info', # 'success', 'info', 'warning', 'danger' or ''
                                        #icon='check', # (FontAwesome names without the `fa-` prefix)
                                        layout=Layout(flex='0 1 auto',
                                        width='auto',)) for g in age_df['Grain/Mica']]



    all_boxes = HBox([remove_grains_w[n] for n in range(0,int(round((age_df['Grain/Mica'].count()))))])
    
    box_layout = Layout(display='flex',
                    flex_flow='row wrap',
                    border='solid 0px',
                    width='100%')
    
    box = Box(children=remove_grains_w, layout=box_layout)
    print("REMOVE GRAINS FROM AGE EQUATION\n   Include\Exclude Grains:")
    display(widgets.HBox([box]))

In [ ]:
if counts_file_w.value != {} or counts_paths != []:
        ###SEE ONLY THOSE GRAINS THAT HAVE SOMETHING IN THE NOTES COLUMN####
    grain_notes = age_df[age_df['Notes'].notnull()]
    grain_notes = grain_notes[["Grain/Mica", "Tracks", "Area(cm2)",
                               "Density(tracks/cm2)", 'Average DPar(µmm)',
                               "U_ppm_m238", "Notes",]]
    print("GRAINS WITH NOTES (for your review):")
    display(grain_notes.style.hide_index())

In [ ]:
if counts_file_w.value != {} or counts_paths != []:    
    print("6) CALCULATE/RE-CALCULATE THE AGE(S)")
    def calc_age(b):
        display(Javascript('IPython.notebook.execute_cells_below()'))
        #display(Javascript('IPython.notebook.execute_cells([14,15,16,17,18,19,20,21])'))
    calc_age_button = widgets.Button(description = "Calculate/Re-calculate Age",
                                    layout=Layout(width='90%'),button_style='success')



    calc_age_button.on_click(calc_age)
    display(calc_age_button)

In [14]:
if counts_file_w.value != {} or counts_paths != []:
    ####DEFINE VARIABLES IN THE DATAFRAM(S) USED TO CLACULATE AGES, ETC..####

    ns = age_df['Tracks']
    area = age_df['Area(cm2)']
    ps = age_df['Tracks']/age_df['Area(cm2)']
    U_ppm = age_df['U_ppm_m238']
    U_ppm_1sigma = age_df['U_ppm_m238_Int2SE']/2
    Th_ppm = age_df['Th_ppm_m232']
    Th_ppm_1sigma = age_df['Th_ppm_m232_Int2SE']/2
    Dpar = age_df['Average DPar(µmm)']
    Dpar_mean = round(np.mean(Dpar),2)
    Dpar_1sigma = age_df['DPar Std Deviation']
    no_grains=ps.count()
    grain_id=age_df['Grain/Mica']
    eU = U_ppm+(0.245*Th_ppm)
    eU_1sigma = U_ppm_1sigma+0.245*Th_ppm_1sigma
    
    if etchant == '5M HNO3'and rmr0pref == '2007':
        rmr0D = round(0.84*((4.58-(0.9231*Dpar+0.2515))/2.98)**(0.21),3)
    elif etchant == '5M HNO3'and rmr0pref == '1999':
        rmr0D = round(1-np.exp(0.647*((0.9231*Dpar+0.2515)-1.75)-1.834),3)
    elif etchant == '5.5M HNO3' and rmr0pref == '2007':
        rmr0D = round(0.84*((4.58-(Dpar))/2.98)**(0.21),3)
    elif etchant == '5.5M HNO3'and rmr0pref == '1999':
        rmr0D = round(1-np.exp(0.647*(Dpar-1.75)-1.834),3)

        #checks if there is a column in the age_df dataframe named "Cl" (if you've added Cl data)
    if 'Cl' in age_df.columns:
        Cl_sem = round(age_df['Cl'],3)
        rmr0 = round(age_df['rmr0'],3)
    else:
        Cl_sem = '--'
    if 'rmr0' in age_df.columns and eCLpref == '2007':
        eCL = round(1-(((rmr0/0.857015)**(1/0.23))+0.13),3)
    elif 'rmr0' in age_df.columns and eCLpref == '1999':
        eCL = round((np.log(1-rmr0)+1.834)/(2.107),3)
    else:
        rmr0 = '--'
        eCL = '--'

    
        #generate SGA
    ri = U_ppm/((area)*((U_ppm_1sigma)**2)) #estimate rho for 0 track grains, re-arrange eq 18 vermeesch 2017, using 0 for Xi error
    ni = (U_ppm/U_ppm_1sigma)**2
        #Handles zero track grains after Vermeesch, 2017
            #in the case of 0 tracks, eq 14 from vermeesch, where his 1/2zeta == Chi
    Age_sga = np.where(ns==0., 1/ld*np.log(1+(Xi)*ld*ri*((ns+0.5)/(ni+0.5))), (1/(ld)*np.log(1+Xi*ld*ps/U_ppm)))
    Age_sga_1sigma = np.where(ns==0., Age_sga*(1/(ns+0.5)+1/(ni+0.5))**(0.5), (1/(ld)*np.log(1+Xi*ld*ps/U_ppm))*(((1/ns)+(U_ppm_1sigma/U_ppm)**(2))**(0.5)))
    ps = np.where(ns==0., ri, ps)

    #Generate Pooled age
    Pooled_age = (1/(ld)*np.log(1+Xi*ld*np.sum(ns)/np.sum(U_ppm*area)))
        #pooled age error
    f_Ti=1/(((2*np.pi)**(0.5))*Age_sga_1sigma/Age_sga)
    f_Ti_k=f_Ti/np.sum(f_Ti)
    Pooled_age_1sigma = (np.sum(((Age_sga-Pooled_age)**2)*f_Ti_k)**0.5)/(no_grains-1)**0.5

    #chi-square test
        #Calculates X2 after Vermeesch, 2017 (same as radial plotter X2)
    zj = np.log(Age_sga)
    sj = Age_sga_1sigma/Age_sga
    X2 = np.sum((zj/sj)**2)-((np.sum(zj/(sj**2)))**2)/(np.sum(1/(sj**2))) #X2 from Vermeesch 2017
    PX2_perc = stats.chi2.sf(X2, no_grains-1)*100

    #central age and dispersion is calculated later

        #create a new dataframe with the ages etc..
            #(sorry for naming standard deviation 4 diff things, cant use same object twice..)
    ns_df = pd.DataFrame({'Grain ID': grain_id, 'Ns': ns, 'Area (cm2)': area,
                          'ps (cm2)': ps, '238U (ppm)': U_ppm, '±':'±','1SD(U)': U_ppm_1sigma,
                          '232U (ppm)': Th_ppm, '±':'±','1SD(Th)': Th_ppm_1sigma,
                           'eU (ppm)': eU,'±2':'±', '1SD(eU)': eU_1sigma, 'Age(Ma)': Age_sga,
                          '±3':'±','1SD(Age)': Age_sga_1sigma,'Dpar (µm)': Dpar,
                          '±4':'±','1SD(Dp)': Dpar_1sigma, 'Cl wt (%)': Cl_sem,
                          'rmr0': rmr0, 'rmr0D': rmr0D, 'eCL': eCL})

    ns_df.reset_index(drop=True, inplace=True)
    #ns_df

In [15]:
if counts_file_w.value != {} or counts_paths != []:
    remove_grs = []
    for i in range(0,age_df['Grain/Mica'].count()):
        if remove_grains_w[i].value == False:
            remove_grs.append(remove_grains_w[i].description)
            #remove the words 'Grain ' from the removed_grain list as this will break other things if left in
            remove_grs = [x.replace('Grain ', '') for x in remove_grs]
    ns_df = ns_df[~ns_df['Grain ID'].isin(remove_grs)]
    
        #removes grains with [U] = 0, aka "Below LOD" that give an age of 'infinity' and break my code :(
    ns_df = ns_df[ns_df['238U (ppm)'] != 0.]
        #removes grains that haven't been counted in FastTracks based on "0" measured area of interest
    ns_df = ns_df[ns_df['Area (cm2)'] != 0.]

    #adds removed grains from above to new list: ns_df_removed
    ns_df_removed = age_df[age_df['Grain/Mica'].isin(remove_grs)]
    ns_df_removed = ns_df_removed.append(age_df[age_df['Area(cm2)'] == 0])
    ns_df_removed = ns_df_removed.append(age_df[age_df['U_ppm_m238'] == 0])
    ns_df_removed.drop_duplicates(subset ="Grain/Mica",keep = 'first', inplace = True)
        #notes reason for removal in column
    ns_df_removed.insert(1, 'Reason for removal', 'User Removed')
    ns_df_removed.loc[ns_df_removed['U_ppm_m238'] == 0, 'Reason for removal'] = "[U] Below detection limit"
    ns_df_removed.loc[ns_df_removed['Area(cm2)'] == 0, 'Reason for removal'] = "Did not measure"
        #select column headers we want; remove unwanted columns
    ns_df_removed = ns_df_removed[['Grain/Mica','Reason for removal','Tracks','U_ppm_m238','Notes']]

        #inspect the grains you've removed, make sure you've removed the correct grains
    print("GRAINS REMOVED FROM AGE EQUATION:")
    display(ns_df_removed.style.hide_index())

In [16]:
if counts_file_w.value != {} or counts_paths != []: 
    ####RE-CALCULATE NEW AGES BASED ON THE REMOVAL OF GRAINS, FINALLY CALCULATES THE CENTRAL AGE####

    ns = ns_df['Ns']
    area = ns_df['Area (cm2)']
    ps = ns_df['Ns']/ns_df['Area (cm2)']
    U_ppm = ns_df['238U (ppm)']
    U_ppm_1sigma = ns_df['1SD(U)']
    Th_ppm = ns_df['232U (ppm)']
    Th_ppm_1sigma = ns_df['1SD(Th)']
    Dpar = ns_df['Dpar (µm)']
    Dpar_mean = round(np.mean(Dpar),2)
    Dpar_1sigma = ns_df['1SD(Dp)']
    no_grains=ps.count()
    grain_id=ns_df['Grain ID']
    eU = ns_df['eU (ppm)']
    eU_1sigma = ns_df['1SD(eU)']
    Cl_sem = ns_df['Cl wt (%)']
    rmr0 = ns_df['rmr0']
    rmr0D = ns_df['rmr0D']
    eCL = ns_df['eCL']

        #checks if there is a column in the age_df dataframe named "Cl" (if you've added Cl data)
    if 'Cl' in age_df.columns:
        Cl_sem_mean = round(np.mean(Cl_sem),3)
        Cl_sem_sdm = round(np.std(Cl_sem), 3)
    else:
        Cl_sem_mean = '--'
        Cl_sem_sdm = '--'
    if 'rmr0' in age_df.columns:
        rmr0_mean = round(np.mean(rmr0),3)
        rmr0_sdm = round(np.std(rmr0), 3)
        eCL_mean = round(np.mean(eCL), 3)
        eCL_sdm = round(np.std(eCL), 3)
    else:
        rmr0_mean = '--'
        rmr0_sdm = '--'
        eCL_mean = '--'
        eCL_sdm = '--'

    rmr0D_mean = round(np.mean(rmr0D),3)
    rmr0D_sdm = round(np.std(rmr0D), 3)

    #generate SGA
    ri = U_ppm/((area)*((U_ppm_1sigma)**2)) #estimate rho for 0 track grains, re-arrange eq 18 vermeesch 2017, using 0 for Xi error
    ni = (U_ppm/U_ppm_1sigma)**2
        #Handles zero track grains after Vermeesch, 2017
            #in the case of 0 tracks, eq 14 from vermeesch, where his 1/2zeta == Chi
    Age_sga = np.where(ns==0., 1/ld*np.log(1+(Xi)*ld*ri*((ns+0.5)/(ni+0.5))), (1/(ld)*np.log(1+Xi*ld*ps/U_ppm)))
    Age_sga_1sigma = np.where(ns==0., Age_sga*(1/(ns+0.5)+1/(ni+0.5))**(0.5), (1/(ld)*np.log(1+Xi*ld*ps/U_ppm))*(((1/ns)+(U_ppm_1sigma/U_ppm)**(2))**(0.5)))
    ps = np.where(ns==0., ri, ps)

    #Pooled age
    Pooled_age = (1/(ld)*np.log(1+Xi*ld*np.sum(ns)/np.sum(U_ppm*area)))
    #pooled age error
    f_Ti=1/(((2*np.pi)**(0.5))*Age_sga_1sigma/Age_sga)
    f_Ti_k=f_Ti/np.sum(f_Ti)
    Pooled_age_1sigma = (np.sum(((Age_sga-Pooled_age)**2)*f_Ti_k)**0.5)/(no_grains-1)**0.5


    #chi-square test
        #Calculates X2 after Vermeesch, 2017 (same as radial plotter X2, different from excel dating sheet)
    zj = np.log(Age_sga)
    sj = Age_sga_1sigma/Age_sga
    X2 = np.sum((zj/sj)**2)-((np.sum(zj/(sj**2)))**2)/(np.sum(1/(sj**2))) #X2 from Vermeesch 2017
    PX2_perc = stats.chi2.sf(X2, no_grains-1)*100
    if PX2_perc <= 4.99:
        chi_test = 'Fail'
    else:
        chi_test = 'Pass'

    ns_df = pd.DataFrame({'Grain ID': grain_id, 'Ns': ns, 'Area (cm2)': area,
                          'ps (cm2)': ps, '238U (ppm)': U_ppm, '±':'±','1SD(U)': U_ppm_1sigma, 
                          '232U (ppm)': Th_ppm, '±':'±','1SD(Th)': Th_ppm_1sigma,
                           'eU (ppm)': eU,'±2':'±', '1SD(eU)': eU_1sigma, 'Age(Ma)': Age_sga,
                          '±3':'±','1SD(Age)': Age_sga_1sigma,'Dpar (µm)': Dpar,
                          '±4':'±','1SD(Dp)': Dpar_1sigma, 'Cl wt (%)': Cl_sem,
                          'rmr0': rmr0, 'rmr0D': rmr0D, 'eCL' : eCL})

    ns_df.reset_index(drop=True, inplace=True)

    #Central age and dispersion
        #uses the CentralAge.py function, should exist in the same folder as this py project.
        #expects 4 arguments in this order CentralAge(grain_ids, single_grain_ages, single_grain_1sigma, pooled_age)
        #grain ids should be something that it can get the total grain count
    C_A = CentralAge(ns_df['Grain ID'], ns_df['Age(Ma)'], ns_df['1SD(Age)'], Pooled_age)
        #returns 3 variables, in this order: Central Age, Central Age 1sigma, Dispersion
    Central_age = C_A[0]
    Central_age_1sigma = C_A[1]
    Dispersion = C_A[2]

    print('Pooled Age:', round(Pooled_age, 2), '±', round(Pooled_age_1sigma,2),'Ma')
    print('Central Age:', round(Central_age, 2), '±', round(Central_age_1sigma, 2),'Ma')
    print('No. of grains: ',grain_id.count())
    print('---------')
    print('X2:', round(X2, 2))
    print('P(X2):', round(PX2_perc, 2),'%')
    print('Dispersion:', round(Dispersion,2),'%')

In [17]:
if counts_file_w.value != {} or counts_paths != []:
    ns_df_sorted = ns_df.sort_values(by='Age(Ma)')
    ns_df_sorted.reset_index(drop=True, inplace=True)
    Age_sga_sorted = ns_df_sorted['Age(Ma)']
    Age_sga_1sigma_sorted = ns_df_sorted['1SD(Age)']
    
    no_grains_sorted=Age_sga_sorted.count()
    grain_id_sorted=ns_df_sorted['Grain ID']

    U_ppm_sorted = ns_df_sorted['238U (ppm)']
    U_ppm_1sigma_sorted = ns_df_sorted['1SD(U)']
    
    eU_ppm_sorted = ns_df_sorted['eU (ppm)']
    eU_ppm_1sigma_sorted = ns_df_sorted['1SD(eU)']
    
    Dpar_sorted = ns_df_sorted['Dpar (µm)']
    Dpar_1sigma_sorted = ns_df_sorted['1SD(Dp)']

    ages_enumerated = []
    for i, ages in enumerate(Age_sga_sorted):
        ages_e = ages/ages+i
        ages_enumerated.append(ages_e)
        
    eU_enumerated = []
    for i, eU_s in enumerate(eU_ppm_sorted):
        eU_e = eU_s/eU_s+i
        eU_enumerated.append(eU_e)
        
    Dpar_enumerated = []
    for i, Dpar_s in enumerate(Dpar_sorted):
        Dpar_e = Dpar_s/Dpar_s+i
        Dpar_enumerated.append(Dpar_e)
        
    # noramlize the color bar based on the mean
    class MidpointNormalize(mpl.colors.Normalize):
        def __init__(self, vmin=None, vmax=None, vcenter=None, clip=False):
            self.vcenter = vcenter
            mpl.colors.Normalize.__init__(self, vmin, vmax, clip)

        def __call__(self, value, clip=None):
            # I'm ignoring masked values
            x, y = [self.vmin, self.vcenter, self.vmax], [0, 0.5, 1]
            return np.ma.masked_array(np.interp(value, x, y))
        
    midnorm_eU = MidpointNormalize(vmin=min(eU_ppm_sorted),
                                   vcenter=np.mean(eU_ppm_sorted), vmax=max(eU_ppm_sorted))
    
    midnorm_Dpar = MidpointNormalize(vmin=min(Dpar_sorted),
                                     vcenter=np.mean(Dpar_sorted), vmax=max(Dpar_sorted))
    
    #lay out the figures using GridSpec
    fig1 = plt.figure(constrained_layout=True, figsize=(16,14))
    spec1 = gs.GridSpec(ncols=3, nrows=2, figure=fig1, width_ratios=(10,0.3,10))

        #upper left figure
    ax = fig1.add_subplot(spec1[0, 0])
        #plot the scatter points with error bars
    ax.hist(Age_sga_sorted, bins = int(grain_id.count()/2),
             color = 'grey', edgecolor = 'black', orientation="horizontal", alpha=0.20)
    eUcolor = ax.scatter(ages_enumerated,Age_sga_sorted, marker='s', c=eU_ppm_sorted, cmap='coolwarm',
               s=140, alpha=0.6, norm=midnorm_eU, zorder=2, label='_nolegend_')
    ax.errorbar(ages_enumerated,Age_sga_sorted, yerr=Age_sga_1sigma_sorted,
                linestyle="None", c='k', alpha=0.2, linewidth=2.5,zorder=1, label='_nolegend_')
        #annotate each plot point with grain id
    for i, txt in enumerate(grain_id_sorted):
        ax.annotate(txt, (ages_enumerated[i], Age_sga_sorted[i]), fontsize=9, weight = 'bold',
                    ha='center', va='center', color='k',alpha=0.8,zorder=3)

    ax.set_yscale('linear') #change to log if you wish
    ax.autoscale(enable=True, axis='y', tight=True)
        #color span for pooled & central age ranges
    ax.axhspan(Pooled_age-Pooled_age_1sigma, Pooled_age+Pooled_age_1sigma, alpha=0.2,
               edgecolor = 'k',facecolor='none',linestyle='--',linewidth=1.5, label='P. Age±SD', zorder=4)
    ax.axhspan(Central_age-Central_age_1sigma, Central_age+Central_age_1sigma, alpha=0.2,
               color='cornflowerblue', label='C. Age±SD', zorder=5)
        #titles and other text
    ax.legend(loc='upper right')
    if ns_df_removed['Grain/Mica'].empty:
        ax.text(0.01, 0.99, 'Pooled Age = {0:.1f} ± {1:.1f}Ma\nCentral Age = {2:.1f} ± {3:.1f}Ma\n χ2 = {4:.1f}\n p(χ2) = {5:.1f}% ({6})\n Disp. = {7:.1f}%\n\n Total Grains = {8}'.format(Pooled_age, Pooled_age_1sigma, Central_age, Central_age_1sigma, X2, PX2_perc, chi_test, Dispersion, grain_id.count()),
                style='italic', horizontalalignment='left', verticalalignment='top', transform=ax.transAxes, fontsize=11)
    else:
        ax.text(0.01, 0.99, 'Pooled Age = {0:.1f} ± {1:.1f}Ma\nCentral Age = {2:.1f} ± {3:.1f}Ma\n χ2 = {4:.1f}\n p(χ2) = {5:.1f}% ({6})\n Disp. = {7:.1f}%\n\n Total Grains = {8}\n Removed GrainID(s):\n{9}'.format(Pooled_age, Pooled_age_1sigma, Central_age, Central_age_1sigma, X2, PX2_perc, chi_test, Dispersion, grain_id.count(),
        ns_df_removed['Grain/Mica'].to_string(index=False)), style='italic', horizontalalignment='left', verticalalignment='top', transform=ax.transAxes, fontsize=11)
    ax.set_title('{0} Single Grain Ages' .format(sample_name))
    ax.set_ylabel('Age (Ma)', fontsize=12)
    ax.set_xlabel('Ascending count based on age', fontsize=12)

    ax3 = plt.subplot(spec1[0, 1])
    c_bar = plt.colorbar(eUcolor, cax=ax3)
    c_bar.set_label("[eU]ppm", fontsize=10)

    ##### upper right figure
    ax2 = fig1.add_subplot(spec1[0, 2])
    euplot = ax2.scatter(x=eU_enumerated,y=eU_ppm_sorted,marker='o', color='white', s=100, alpha=1, zorder=2)
    ax2.errorbar(x=eU_enumerated,y=eU_ppm_sorted, yerr=eU_ppm_1sigma_sorted, linestyle="None", linewidth=2.5,color='k', alpha=0.2, zorder=1)
    ax2.grid(which='major', axis='y', alpha=0.6)    
    ax2.grid(which='minor', axis='y', alpha=0.2)   
        #annotate each plot point with grain number
    for i, txt in enumerate(grain_id_sorted):
        ax2.annotate(txt, (eU_enumerated[i], eU_ppm_sorted[i]), fontsize=9, weight = 'bold', ha='center', va='center', color='#942222', zorder=3)
    #for i,j in zip(grain_id,eU):
    #    ax2.annotate(str(i),xy=(i,j), fontsize=9, weight = 'bold', ha='center', va='center', color='#90033C', zorder=3)
    ax2.set_yscale('log')
    ax2.yaxis.set_label_position("right")
    ax2.yaxis.tick_right()
        #color span for u±1sigma
    ax2.axhspan(np.mean(eU)-np.std(eU), 
                np.mean(eU)+np.std(eU), 
                alpha=0.2, color='cornflowerblue')
        #draw line at less than 1 ppm, consider removing these grains (anonymously old usualy)
    ax2.axhline(y=1,linewidth=1, linestyle='--', color='#970000')
    ax2.axhspan(1, 0, alpha=0.1, color='r')
        #plot text that appears inside the plot
    ax2.text(min(i for i in eU_enumerated if not math.isnan(i)), (np.mean(eU)+np.std(eU))*0.80, 
             'mean [eU]: {0:.2f}±{1:.1f}ppm'.format(np.mean(eU),np.std(eU)), 
             style='italic', horizontalalignment='left', fontsize=10, color='cornflowerblue')

        #titles and other text
    #ax2.legend(loc='upper right')
    ax2.set_title('{0} [eU] ppm' .format(sample_name))
    ax2.set_ylabel('[eU] ppm', fontsize=12)
    ax2.set_xlabel('Ascending count based on age', fontsize=12)

    #######################################
    #plot age vs cl info
    if 'Cl' in age_df.columns:
        eCL_sorted = ns_df_sorted['eCL']
        #eCL_1sigma_sorted = ns_df_sorted['1SD(Dp)']
        midnorm_eCL = MidpointNormalize(vmin=min(eCL_sorted), vcenter=np.mean(eCL_sorted), vmax=max(eCL_sorted))
        #eCL_enumerated = []
        #for i, eCL_s in enumerate(eCL_sorted):
        #    eCL_e = eCL_s/eCL_s+i
        #    eCL_enumerated.append(eCL_e)
            #lower left figure
        ax4 = fig1.add_subplot(spec1[1, 0])
            #plot the scatter points with error bars
        ax4.hist(Age_sga_sorted, bins = int(grain_id.count()/2),
                 color = 'grey', edgecolor = 'black', orientation="horizontal", alpha=0.20)
        cmap = plt.get_cmap('PiYG')
        cmap.set_bad("yellow")
        eCLcolor = ax4.scatter(ages_enumerated,Age_sga_sorted, marker='s', c=eCL_sorted, cmap=cmap,
                    plotnonfinite=True, vmin=np.min(eCL_sorted), vmax=np.max(eCL_sorted),
                   s=140, alpha=0.6, norm=midnorm_eCL, zorder=2, label='_nolegend_')
        ax4.errorbar(ages_enumerated,Age_sga_sorted, yerr=Age_sga_1sigma_sorted,
                    linestyle="None", c='k', alpha=0.2, linewidth=2.5,zorder=1, label='_nolegend_')
            #annotate each plot point with grain id
        for i, txt in enumerate(grain_id_sorted):
            ax4.annotate(txt, (ages_enumerated[i], Age_sga_sorted[i]), fontsize=9, weight = 'bold',
                        ha='center', va='center', color='k',alpha=0.8,zorder=3)

        ax4.set_yscale('linear') #change to log if you wish
        ax4.autoscale(enable=True, axis='y', tight=True)
            #color span for pooled & central age ranges
        ax4.axhspan(Pooled_age-Pooled_age_1sigma, Pooled_age+Pooled_age_1sigma, alpha=0.2,
                   edgecolor = 'k',facecolor='none',linestyle='--',linewidth=1.5, label='P. Age±SD', zorder=4)
        ax4.axhspan(Central_age-Central_age_1sigma, Central_age+Central_age_1sigma, alpha=0.2,
                   color='cornflowerblue', label='C. Age±SD', zorder=5)
            #titles and other text
        ax4.legend(loc='upper right')
        if ns_df_removed['Grain/Mica'].empty:
            ax4.text(0.01, 0.99, 'Pooled Age = {0:.1f} ± {1:.1f}Ma\nCentral Age = {2:.1f} ± {3:.1f}Ma\n χ2 = {4:.1f}\n p(χ2) = {5:.1f}% ({6})\n Disp. = {7:.1f}%\n\n Total Grains = {8}'.format(Pooled_age, Pooled_age_1sigma, Central_age, Central_age_1sigma, X2, PX2_perc, chi_test, Dispersion, grain_id.count()),
                    style='italic', horizontalalignment='left', verticalalignment='top', transform=ax4.transAxes, fontsize=11)
        else:
            ax4.text(0.01, 0.99, 'Pooled Age = {0:.1f} ± {1:.1f}Ma\nCentral Age = {2:.1f} ± {3:.1f}Ma\n χ2 = {4:.1f}\n p(χ2) = {5:.1f}% ({6})\n Disp. = {7:.1f}%\n\n Total Grains = {8}\n Removed GrainID(s):\n{9}'.format(Pooled_age, Pooled_age_1sigma, Central_age, Central_age_1sigma, X2, PX2_perc, chi_test, Dispersion, grain_id.count(),
            ns_df_removed['Grain/Mica'].to_string(index=False)), style='italic', horizontalalignment='left', verticalalignment='top', transform=ax4.transAxes, fontsize=11)
        ax4.set_title('{0} Single Grain Ages' .format(sample_name))
        ax4.set_ylabel('Age (Ma)', fontsize=12)
        ax4.set_xlabel('Ascending count based on age', fontsize=12)

        ax5 = plt.subplot(spec1[1, 1])
        c_bar = plt.colorbar(eCLcolor, cax=ax5)
        c_bar.set_label("eCL (apfu)", fontsize=10)

        ##### lower right figure
        ax6 = fig1.add_subplot(spec1[1, 2])
        Dparplot = ax6.scatter(x=ages_enumerated,y=eCL_sorted,marker='o', color='white', s=100, alpha=1, zorder=2)
        ax6.grid(which='major', axis='y', alpha=0.5, linestyle="--")    
        #ax6.grid(which='minor', axis='y', alpha=0.2)   
            #annotate each plot point with grain number
        for i, txt in enumerate(grain_id_sorted):
            ax6.annotate(txt, (ages_enumerated[i], eCL_sorted[i]), fontsize=9,
                         weight = 'bold', ha='center', va='center', color='#942222', zorder=3)

        ax6.yaxis.set_label_position("right")
        ax6.yaxis.tick_right()
            #color span for dpar±1sigma
        ax6.axhspan(np.mean(eCL)-np.std(eCL), np.mean(eCL)+np.std(eCL),
                    alpha=0.2, color='cornflowerblue')
            #plot text that appears inside the plot
        ax6.text(min(i for i in ages_enumerated if not math.isnan(i)), np.mean(eCL)+np.std(eCL)*0.8, 
                 'mean eCl: {0:.3f}±{1:.3f} apfu'.format(np.mean(eCL),np.std(eCL)) ,
                 style='italic', horizontalalignment='left', fontsize=10, color='cornflowerblue')

            #titles and other text
        #ax6.legend(loc='upper right')
        ax6.set_title('{0} eCl (apfu)' .format(sample_name))
        ax6.set_ylabel('eCl (apfu)', fontsize=12)
        ax6.set_xlabel('Ascending count based on age', fontsize=12)
        
       #plot using Dpar if Cl doesnt exisst 
    else:
            #lower left figure
        ax4 = fig1.add_subplot(spec1[1, 0])
            #plot the scatter points with error bars
        ax4.hist(Age_sga_sorted, bins = int(grain_id.count()/2),
                 color = 'grey', edgecolor = 'black', orientation="horizontal", alpha=0.20)
        cmap = plt.get_cmap('PiYG')
        cmap.set_bad("yellow")
        Dparcolor = ax4.scatter(ages_enumerated,Age_sga_sorted, marker='s', c=Dpar_sorted, cmap=cmap,
                    plotnonfinite=True, vmin=np.min(Dpar_sorted), vmax=np.max(Dpar_sorted),
                   s=140, alpha=0.6, norm=midnorm_Dpar, zorder=2, label='_nolegend_')
        ax4.errorbar(ages_enumerated,Age_sga_sorted, yerr=Age_sga_1sigma_sorted,
                    linestyle="None", c='k', alpha=0.2, linewidth=2.5,zorder=1, label='_nolegend_')
            #annotate each plot point with grain id
        for i, txt in enumerate(grain_id_sorted):
            ax4.annotate(txt, (ages_enumerated[i], Age_sga_sorted[i]), fontsize=9, weight = 'bold',
                        ha='center', va='center', color='k',alpha=0.8,zorder=3)

        ax4.set_yscale('linear') #change to log if you wish
        ax4.autoscale(enable=True, axis='y', tight=True)
            #color span for pooled & central age ranges
        ax4.axhspan(Pooled_age-Pooled_age_1sigma, Pooled_age+Pooled_age_1sigma, alpha=0.2,
                   edgecolor = 'k',facecolor='none',linestyle='--',linewidth=1.5, label='P. Age±SD', zorder=4)
        ax4.axhspan(Central_age-Central_age_1sigma, Central_age+Central_age_1sigma, alpha=0.2,
                   color='cornflowerblue', label='C. Age±SD', zorder=5)
            #titles and other text
        ax4.legend(loc='upper right')
        if ns_df_removed['Grain/Mica'].empty:
            ax4.text(0.01, 0.99, 'Pooled Age = {0:.1f} ± {1:.1f}Ma\nCentral Age = {2:.1f} ± {3:.1f}Ma\n χ2 = {4:.1f}\n p(χ2) = {5:.1f}% ({6})\n Disp. = {7:.1f}%\n\n Total Grains = {8}'.format(Pooled_age, Pooled_age_1sigma, Central_age, Central_age_1sigma, X2, PX2_perc, chi_test, Dispersion, grain_id.count()),
                    style='italic', horizontalalignment='left', verticalalignment='top', transform=ax4.transAxes, fontsize=11)
        else:
            ax4.text(0.01, 0.99, 'Pooled Age = {0:.1f} ± {1:.1f}Ma\nCentral Age = {2:.1f} ± {3:.1f}Ma\n χ2 = {4:.1f}\n p(χ2) = {5:.1f}% ({6})\n Disp. = {7:.1f}%\n\n Total Grains = {8}\n Removed GrainID(s):\n{9}'.format(Pooled_age, Pooled_age_1sigma, Central_age, Central_age_1sigma, X2, PX2_perc, chi_test, Dispersion, grain_id.count(),
            ns_df_removed['Grain/Mica'].to_string(index=False)), style='italic', horizontalalignment='left', verticalalignment='top', transform=ax4.transAxes, fontsize=11)
        ax4.set_title('{0} Single Grain Ages' .format(sample_name))
        ax4.set_ylabel('Age (Ma)', fontsize=12)
        ax4.set_xlabel('Ascending count based on age', fontsize=12)

        ax5 = plt.subplot(spec1[1, 1])
        c_bar = plt.colorbar(Dparcolor, cax=ax5)
        c_bar.set_label("Dpar (µm)", fontsize=10)

        ##### lower right figure
        ax6 = fig1.add_subplot(spec1[1, 2])
        Dparplot = ax6.scatter(x=Dpar_enumerated,y=Dpar_sorted,marker='o', color='white',
                              s=100, alpha=1, zorder=2)
        ax6.errorbar(x=Dpar_enumerated,y=Dpar_sorted, yerr=Dpar_1sigma_sorted, linestyle="None",
                     linewidth=2.5,color='k', alpha=0.2, zorder=1)
        ax6.grid(which='major', axis='y', alpha=0.5, linestyle="--")    
        #ax6.grid(which='minor', axis='y', alpha=0.2)   
            #annotate each plot point with grain number
        for i, txt in enumerate(grain_id_sorted):
            ax6.annotate(txt, (Dpar_enumerated[i], Dpar_sorted[i]), fontsize=9,
                         weight = 'bold', ha='center', va='center', color='#942222', zorder=3)

        ax6.yaxis.set_label_position("right")
        ax6.yaxis.tick_right()
            #color span for dpar±1sigma
        ax6.axhspan(np.mean(Dpar)-np.mean(Dpar_1sigma), np.mean(Dpar)+np.mean(Dpar_1sigma),
                    alpha=0.2, color='cornflowerblue')
            #plot text that appears inside the plot
        ax6.text(min(i for i in Dpar_enumerated if not math.isnan(i)), 
                 np.mean(Dpar)+np.mean(Dpar_1sigma)*0.8, 'mean Dpar: {0:.2f}±{1:.1f} µm'.format(np.mean(Dpar),np.mean(Dpar_1sigma)) ,
                 style='italic', horizontalalignment='left', fontsize=10, color='cornflowerblue')

            #titles and other text
        #ax6.legend(loc='upper right')
        ax6.set_title('{0} Dpar (µm)' .format(sample_name))
        ax6.set_ylabel('Dpar (µm)', fontsize=12)
        ax6.set_xlabel('Ascending count based on age', fontsize=12)


        #save the figure to file, location defined at the start
    plt.savefig("{0}/{1}_GrainvAge_eU.pdf".format(save_to, sample_name), bbox_inches='tight', transparent=True)

In [18]:
if counts_file_w.value != {} or counts_paths != []:
    #Finally, save and export summary table and other sheets to import into modelling prog.
    
    #modify variables depending on if found in FastTracks file or inserted in text boxes at start
    if sample_name_w.value == '':
        sample_name = sample_FT
    
    if igsn_w.value == '':
        igsn = igsn_FT
    if igsn_FT == '':
        igsn = '--'
    
    if analyst_w.value == '':
        analyst = analyst_FT

    #Create summary table to single grain ages:
        #header
    single_grain_header = pd.DataFrame({'col0':['Sample:','IGSN:','Mineral:', 'Rock Type:', 
                                                'Lat/Long:', 'Elevation:', '','Grain ID'],
                                       'col1':['', '', '', '', '', '','','Ns'],
                                       'col2':[sample_name,igsn,mineral,rock_type,'{0:.5f}'.format(latitude),
                                               '{0} m'.format(elevation),'','Area (cm2)'],
                                       'col3':['', '', '', '','/{0:.5f}'.format(longitude), '','','ρS (cm-2)'],
                                       'col4':['', '', '', '', '', '','','[U] (ppm)'],
                                       'col5':['', '', '', '', '', '','','±'],
                                       'col6':['', '', '', '', '', '','','1σ'],
                                       'col7':['', '', '', '', '', '','','[Th] (ppm)'],
                                       'col8':['', '', '', '', '', '','','±'],
                                       'col9':['', '', '', '', '', '','','1σ'],
                                       'col10':['', '', '', '', '', '','','e[U] (ppm)'],
                                       'col11':['', '', '', '', '', '','','±'],
                                       'col12':['', '', '', '', '', '','','1σ'],
                                       'col13':['Analyst:', 'Collector:', 'Date Meas.:', 'Software:', 
                                                'U Stand.:', 'Int. Stand.:','','Age (Ma)'],
                                       'col14':['', '', '', '', '', '','','±'],
                                       'col15':[analyst,collector,date_meas.strftime("%d/%m/%Y"),
                                                software,Ustandard,
                                                '{0}, w/ mean [U]: {1}'" "u'\xb1'" "'{2}*'" "'ppm' .format(Intstandard,mean_intstd,meanerr_intstd),'','1σ'],
                                       'col16':['', '', '', '', '', '','','Dpar (µm)'],
                                       'col17':['', '', '', '', '', '','','±'],
                                       'col18':['', '', '', '', '', '','','SE'],
                                       'col19':['', '', '', '', '','','','rmr0†'],
                                       'col20':['', '', '', '', '', '','','rmr0\u1D30‡'],
                                       'col21':['', '', '', '', '', '','','Cl (wt%)'],
                                       'col22':['', '', '', '', '', '','','eCl (apfu)§']})

        #body of single grain ages
    single_grain_body = pd.DataFrame({'col0':grain_id,
                                        'col1':ns.astype(int),
                                        'col2':area,
                                        'col3':ps.round(2),
                                        'col4':U_ppm.round(2),
                                        'col5':'±',
                                        'col6':U_ppm_1sigma.round(2),
                                        'col7':Th_ppm.round(2),
                                        'col8':'±',
                                        'col9':Th_ppm_1sigma.round(2),
                                        'col10':eU.round(2),
                                        'col11':'±',
                                        'col12':eU_1sigma.round(2),
                                        'col13':Age_sga.round(1),
                                        'col14':'±',
                                        'col15':Age_sga_1sigma.round(1),
                                        'col16':Dpar.round(2),
                                        'col17':'±',
                                        'col18':Dpar_1sigma.round(2),
                                        'col19':rmr0,
                                        'col20':rmr0D,
                                        'col21':Cl_sem,
                                        'col22':eCL.replace(-0.000,0.000),})
    single_grain_body = single_grain_body.replace({"col3":{0:np.nan}})
    single_grain_body = single_grain_body.replace({"col18":{0:np.nan}})
    
    #variables for the footer based on choices at the start
    if eCLpref == '2007' and rmr0pref == '2007':
        footer_pref = '*std. dev. of mean,   †rmr0 proxy following Ketcham et al. (2007),  ‡rmr0\u1D30 est. from Dpar (Ketcham et al., 2007),   §effective Cl (Ketcham et al., 2007, McDannell et al., 2019)'
    elif eCLpref == '1999' and rmr0pref == '1999':
        footer_pref = '*std. dev. of mean,   †rmr0 proxy following Ketcham et al. (1999),  ‡rmr0\u1D30 est. from Dpar (Ketcham et al., 1999),   §effective Cl (Ketcham et al., 1999, McDannell et al., 2019)'
    elif eCLpref == '2007' and rmr0pref == '1999':
        footer_pref = '*std. dev. of mean,   †rmr0 proxy following Ketcham et al. (1999),  ‡rmr0\u1D30 est. from Dpar (Ketcham et al., 1999),   §effective Cl (Ketcham et al., 2007, McDannell et al., 2019)'
    elif eCLpref == '1999' and rmr0pref == '2007':
        footer_pref = '*std. dev. of mean,   †rmr0 proxy following Ketcham et al. (2007),  ‡rmr0\u1D30 est. from Dpar (Ketcham et al., 2007),   §effective Cl (Ketcham et al., 1999, McDannell et al., 2019)'
        #footer of single grain ages
    single_grain_footer = pd.DataFrame({'col0':['',ns.count(), '{0}'.format(footer_pref), '', 'χ2', 'p(χ2)', 'Dispersion'],
                                       'col1':['',np.sum(ns).astype(int), '', '', '=', '=', '='],
                                       'col2':['',"{:.3E}".format(np.sum(area)), '', '',"{:.1f}".format(X2),
                                               "{:.1f}%".format(PX2_perc),"{:.1f}%".format(Dispersion)],
                                       'col3':['',"{:.3E}".format(np.mean(ps)), '', '', '', '({0})'.format(chi_test), ''],
                                       'col4':['',"{:.1f}".format(np.mean(U_ppm)), '', '', '', '', ''],
                                       'col5':['','±', '', '', '', '', ''],
                                       'col6':['',"{:.1f}*".format(np.std(U_ppm)), '', '', '', '', ''],
                                       'col7':['',"{:.1f}".format(np.mean(Th_ppm)), '', '', '', '', ''],
                                       'col8':['','±', '', '', '', '', ''],
                                       'col9':['',"{:.1f}*".format(np.std(Th_ppm)), '', '', '', '', ''],
                                       'col10':['',"{:.1f}".format(np.mean(eU)), '', '', '', '', ''],
                                       'col11':['','±', '', '', '', 'Pooled Age =', 'Central Age ='],
                                       'col12':['',"{:.1f}*".format(np.std(eU)), '', '', '', '', ''],
                                       'col13':['','', '', '', '',  "{:.1f}".format(Pooled_age),
                                                "{:.1f}".format(Central_age)],
                                       'col14':['','', '', '', '', '±', '±'],
                                       'col15':['','','', '', '', "{:.1f}".format(Pooled_age_1sigma),
                                                "{:.1f}".format(Central_age_1sigma)],
                                       'col16':['',"{:.2f}".format(np.mean(Dpar)), '', '', '','',''],
                                       'col17':['','±', '', '', '', '',''],
                                       'col18':['',"{:.2f}*".format(np.std(Dpar)), '', '', '','',''],
                                       'col19':['',rmr0_mean, '', '', '', '', ''],
                                       'col20':['',rmr0D_mean, '', '', '', '', ''],
                                       'col21':['',Cl_sem_mean, '', '', '', '', ''],
                                       'col22':['',eCL_mean, '', '', '', '', '']})

    #bin the length data
    length_range=[0,1,2,4,5,6,7,8,9,10,11,12,13,14,15,16,17]
    if mtl != '--':
        lengths_summary = lengths_data['True Length'].groupby(pd.cut(lengths_data['True Length'], length_range)).count()
        lengths_binned = pd.DataFrame ({"L1":['','','','','','','','Length (µm)','0-1','1-2','2-3','3-4','4-5','5-6',
                        '6-7','7-8','8-9','9-10','10-11','11-12',
                        '12-13','13-14','14-15','15-16','16-17','','MTL (µm) =','Var =','SD =','No. =','','Dpar (avg) =','rmr0\u1D30 ='],
                        "L2":['','','','','','','','Count',lengths_summary[1],lengths_summary[2],lengths_summary[3],
                        lengths_summary[4],lengths_summary[5],lengths_summary[6],lengths_summary[7],lengths_summary[8],
                        lengths_summary[9],lengths_summary[10],lengths_summary[11],lengths_summary[12],lengths_summary[13],
                        lengths_summary[14],lengths_summary[15],lengths_summary[16],lengths_summary[17],
                        '',"{:.2f}".format(mtl),
                        "{:.2f}".format(mtl_var)
                        ,"{:.2f}".format(mtl_sd),l_no,'',Dpar_lengths_mean,rmr0D_lengths_mean]}) 
    else:
        lengths_binned = pd.DataFrame ({"L1":[''],"L2":['']})

    #combine the above dataframes horizontally (stack)
    single_grain_data = pd.concat([single_grain_header, single_grain_body, single_grain_footer], axis=0)
    #add the length bins to the right side
    single_grain_data = pd.concat([single_grain_data.reset_index(drop=1),lengths_binned.reset_index(drop=1)], axis=1).fillna('')

    #export a csv of summary body only data for possible later use
    single_grain_sim = single_grain_body
    single_grain_sim = single_grain_sim[["col0","col1","col2","col3","col4","col6",
                                         "col7","col9","col10","col12","col13","col15",
                                         "col16","col18","col19","col20","col21","col22"]] #use only these, omit ± cols.
    single_grain_sim.columns = ["grain","ns","area","ps","uppm","uppm_sd","thppm","thppm_sd","euppm",
                                "euppm_sd","age","age_sd","dpar","dpar_se","rmr0","rmr0D","cl_wt","eCl"]
    single_grain_sim = pd.DataFrame(single_grain_sim)
    single_grain_sim.reset_index(drop=True, inplace=True)
    single_grain_sim.loc[0,'sample_no'] = sample_name 
    single_grain_sim.loc[0,'central_age'] = Central_age 
    single_grain_sim.loc[0,'central_age_sd'] = Central_age_1sigma
    single_grain_sim.loc[0,'pooled_age'] = Pooled_age 
    single_grain_sim.loc[0,'pooled_age_sd'] = Pooled_age_1sigma
    single_grain_sim.loc[0,'chi_2'] = X2
    single_grain_sim.loc[0,'pchi'] = PX2_perc 
    single_grain_sim.loc[0,'dispersion'] = Dispersion
    single_grain_sim.to_csv('{0}/{1}_age_summary.csv'.format(save_to, sample_name), index=None)

    #export a csv of summary of lengths data for possible later use
    if mtl != '--':
        lengths_sim = lengths_data[["Length Name", "True Length"]]
        lengths_sim.columns = ["length_no","true_length"]
        lengths_sim = pd.DataFrame(lengths_sim)
        lengths_sim.reset_index(drop=True, inplace=True)
        lengths_sim['dpar_avg'] = Dpar_lengths
        lengths_sim['rmr0'] = rmr0D_lengths
        lengths_sim.to_csv('{0}/{1}_lengths_summary.csv'.format(save_to, sample_name), index=None)
    ###############################################################################################################
    #Create variables table
    variables_table = pd.DataFrame({'Variable':["M[238U]","N₀","d(Ap)","R(Ap)","q(Ap)","λd","λf","____","ξ"],
                                   'Value':[M238U,No,dAp,"{:.3e}".format(RAp),qAp,ld,lf,"","{:.4e}".format(Xi)],
                                    'Unit':["g/mol","—","g/cm^3","g/cm^3","—","1/t","1/t","","tcm^2"],
                                    'Comment':["Molar Mass 238U","Avogadros No.",
                                              "Density of apatite. derived from a calculated relationship between density and apatite Cl content based on the analyses and unit cell dimensions of Carlson et al. (1999)",
                                              "Based on half the confined track length for spontaneous tracks apatites (7.5 for volcanic apatites, see (Gleadow et al., 1986); 7.17 average of Durango).",
                                              "The relatively few direct measurements of this efficiency factor range from 0.90–0.99 (e.g. Iwano et al., 1993; Jonckheere and Van den haute, 2002). Hasebe et al. (2004) used a value of 1.0. Clearly more experimental work is needed to define this parameter with the counting setup used, but for this discussion a value of 0.96 is used.",
                                              "λd",
                                              "Spontaneous fission decay constant, value from Yoshioka et al. (2005)",
                                              "","Aggregate factor = M238U/(λf*N₀*dAp*RAp*qAp)"]})

    ###########################################################
    #Combine all above into 1 xlsx file with multiple sheets 
    writer = pd.ExcelWriter('{0}/{1}_Summary.xlsx'.format(save_to, sample_name),  
                              engine ='xlsxwriter',options={'strings_to_urls': False, 
                                     'strings_to_formulas': False})
    single_grain_data.to_excel(writer, sheet_name ='Ages', header = False, index = False)
    #ns_df_removed.loc[no_grains+100,'U_ppm_m238'] = "Copy/paste to retrieve:"
    #ns_df_removed.loc[no_grains+100,'Notes'] = str(remove_grs)
    ns_df_removed.to_excel(writer, sheet_name ='Removed_Grains', header = True, index = False)
    variables_table.to_excel(writer, sheet_name ='Variables', header = True, index = False)
    single_grain_sim.to_excel(writer, sheet_name ='Single_Grain_Body', header = True, index = False)
    counts_data.to_excel(writer, sheet_name ='Raw_Count_Data', header = True, index = False)
    icpms_data.to_excel(writer, sheet_name ='Raw_ICPMS_Data', header = True, index = False)
    if mtl != '--':
        lengths_data.to_excel(writer, sheet_name ='Raw_Length_Data', header = True, index = False)

    workbook = writer.book
    worksheet = writer.sheets['Ages'] 

    sci_format = workbook.add_format({'num_format': '0.000E+00'})
    format_boldC = workbook.add_format({'bold':  True, 'align': 'center'})
    format_boldCTR = workbook.add_format({'bold':  True, 'align': 'center','text_wrap': True})
    format_boldL = workbook.add_format({'bold':  True, 'align': 'left'})
    format_center = workbook.add_format({'align': 'center'})
    format_right = workbook.add_format({'align': 'right'})
    format_left = workbook.add_format({'align': 'left'})
    format_wrap = workbook.add_format({'align': 'left', 'valign': 'top', 'text_wrap': True})
        #modify columns
    worksheet.set_column('B:B', 5, cell_format=format_center)#ns
    worksheet.set_column('C:D', cell_format=sci_format)#area and rhos
    worksheet.set_column('E:E', 6, cell_format=format_right)#Uppm
    worksheet.set_column('F:F', 2, cell_format=format_center)#±
    worksheet.set_column('G:G', 5, cell_format=format_left)#U ppm sd
    worksheet.set_column('H:H', 6, cell_format=format_right)#Thppm
    worksheet.set_column('I:I', 2, cell_format=format_center)#±
    worksheet.set_column('J:J', 5, cell_format=format_left)#Thppm sd
    worksheet.set_column('K:K', 6, cell_format=format_right)#Eu ppm
    worksheet.set_column('L:L', 2, cell_format=format_center)#±
    worksheet.set_column('M:M', 5, cell_format=format_left)#Euppm sd
    worksheet.set_column('N:N', 8, cell_format=format_right)# age
    worksheet.set_column('O:O', 2, cell_format=format_center)# ±
    worksheet.set_column('P:P', 5, cell_format=format_left)# age sd
    worksheet.set_column('Q:Q', 5, cell_format=format_right)# Dpar
    worksheet.set_column('R:R', 2, cell_format=format_center)# ±
    worksheet.set_column('S:S', 4, cell_format=format_left)#Dpar sd
    worksheet.set_column('T:W', 7, cell_format=format_center)#rmr0, rmr0D, CLwt, eCl
    worksheet.set_column('X:X', 10, cell_format=format_center)#Length
    worksheet.set_column('Y:Y', 8, cell_format=format_center)#Count


        #modify rows
    worksheet.set_row((0),20, cell_format=format_boldL)
    worksheet.set_row((1), cell_format=format_left)
    worksheet.set_row((2), cell_format=format_left)
    worksheet.set_row((3), cell_format=format_left)
    worksheet.set_row((4), cell_format=format_left)
    worksheet.set_row((5), cell_format=format_left)
    worksheet.set_row((6), cell_format=format_left)
    worksheet.set_row((7), 28, cell_format=format_boldCTR)
    worksheet.set_row((no_grains+9), 20, cell_format=format_boldC)
    worksheet.set_row((no_grains+12), cell_format=format_boldC)
    worksheet.set_row((no_grains+13), cell_format=format_boldC)
    worksheet.set_row((no_grains+14), cell_format=format_boldC)

    workbook2 = writer.book
    worksheet2 = writer.sheets['Variables'] 
    format_wrap2 = workbook2.add_format({'align': 'left', 'valign': 'top', 'text_wrap': True})
    worksheet2.set_column('D:D', 100, cell_format=format_wrap2)

    workbook3 = writer.book
    worksheet3 = writer.sheets['Raw_Count_Data'] 
    format_wrap3 = workbook3.add_format({'align': 'left', 'valign': 'top', 'text_wrap': True})
    worksheet3.set_column('G:G', 80, cell_format=format_wrap3)
    writer.save()
    ###############################################################################################################
    #create a csv that will append pooled data, this will append new data (ie: next sample) as a new row so you can use the bulk data in other programs
    append_pooled_data = pd.DataFrame({"date_created":datetime.now().strftime("%H:%M:%S %d/%m/%Y"),
                                        "sample_name":[sample_name],
                                       "igsn":[igsn],
                                       "collector":[collector],
                                       "analyst":[analyst],
                                       "mineral":[mineral],
                                       "rock_type":[rock_type],
                                       "country":[country],
                                       "region":[region],
                                       "latitude":[latitude],
                                       "longitude":[longitude],
                                       "elevation_m":[elevation],
                                       "Ustandard":[Ustandard],
                                       "IntU_standard":[Intstandard],
                                       "IntU_mean":[mean_intstd],
                                       "IntU_sd":[meanerr_intstd],
                                       "spot_size":[spot_size],
                                       "lab_name":[lab_name],
                                       "etchant":[etchant],
                                       "etching_time":[etching_time],
                                       "etching_temp":[etching_temp],
                                       "Xi":[round(Xi, 7)],
                                       "grains_tot":[ns.count()],
                                       "ns_tot":[np.sum(ns).astype(int)],
                                       "area_tot":[round(np.sum(area),10)],
                                       "ps_mean":[round(np.mean(ps),2)],
                                       "uppm_mean":[round(np.mean(U_ppm),2)],
                                       "uppm_sdm":[round(np.std(U_ppm), 2)],
                                       "thppm_mean":[round(np.mean(Th_ppm),2)],
                                       "thppm_sdm":[round(np.std(Th_ppm), 2)],
                                       "euppm_mean":[round(np.mean(eU), 2)],
                                       "euppm_sdm":[round(np.std(eU), 2)],
                                       "page":[round(Pooled_age, 2)],
                                       "page_sd":[round(Pooled_age_1sigma, 2)],
                                       "cage":[round(Central_age, 2)],
                                       "cage_sd":[round(Central_age_1sigma, 2)],
                                       "X2":[round(X2, 2)],
                                       "PX2":[round(PX2_perc, 2)],
                                       "dispersion":[round(Dispersion, 2)],
                                       "dpar_mean":[round(np.mean(Dpar), 2)],
                                       "dpar_sdm":[round(np.std(Dpar), 2)],
                                       "cl_wt_mean":[np.where(Cl_sem_mean == '--', '', Cl_sem_mean)],
                                       "cl_sdm":[np.where(Cl_sem_sdm == '--', '', Cl_sem_sdm)],
                                       "rmr0_mean":[np.where(rmr0_mean == '--', '', rmr0_mean)],
                                       "rmr0_sdm":[np.where(rmr0_sdm == '--', '', rmr0_sdm)],
                                       "rmr0D_mean":[np.where(rmr0D_mean == '--', '', rmr0D_mean)],
                                       "rmr0D_sdm":[np.where(rmr0D_sdm == '--', '', rmr0D_sdm)],
                                       "len_no":[np.where(l_no == '--', '', l_no)],
                                       "mtl":[np.where(mtl == '--', '', mtl)],
                                       "mtl_var":[np.where(mtl_var == '--', '', mtl_var)],
                                       "mtl_sd":[np.where(mtl_sd == '--', '', mtl_sd)],
                                       "mtl_dpar_mean":[Dpar_lengths_mean],
                                       "rmr0D_length_mean":[rmr0D_lengths_mean],
                                       "rmr0D_length_sdm":[rmr0D_lengths_sdm],
                                       "eCl_mean":[np.where(eCL_mean == '--', '', eCL_mean)],
                                       "eCl_sdm":[np.where(eCL_sdm == '--', '', eCL_sdm)]})

    up_onefolder = str(Path(save_to).parents[0])
    append_filepath = Path('{0}/pooled_age_summary.csv'.format(up_onefolder))

    if append_filepath.exists():
        append_pooled_df = pd.read_csv('{0}/pooled_age_summary.csv'.format(up_onefolder)) 
        append_pooled_df.drop(append_pooled_df[append_pooled_df['sample_name'] == sample_name].index,
                              inplace=True, errors='raise')
        append_pooled_df.to_csv('{0}/pooled_age_summary.csv'.format(up_onefolder), header=True, index=None)
        append_pooled_data.to_csv('{0}/pooled_age_summary.csv'.format(up_onefolder), mode='a', header=False, index=None)
    else:
        append_pooled_data.to_csv('{0}/pooled_age_summary.csv'.format(up_onefolder), header=True, index=None)
    ###############################################################################################################
    #create and export tables for use in various programs
        #create a folder to store these
    data_folder = '{0}/Data_Import'.format(save_to)
    if not os.path.exists(data_folder): os.makedirs(data_folder)
    #export tables to use in HeFty
        #Age File:
        ###using dpar
    hefty_age_header = pd.DataFrame({sample_name:["Zeta:","zeta","4352.32","Ns"],
                                    "col1":["LAICPMS ratio","sig zeta","1.1","Area (cm²)"],
                                    "col2":["","","","Pcorr"],
                                    "col3":["","","","sig(Pcorr)"],
                                    "col4":["","","","Dpar"]})   
    hefty_age_data = pd.DataFrame({sample_name:ns,
                                     "col1":area,
                                     "col2":U_ppm,
                                     "col3":U_ppm_1sigma,
                                     "col4":Dpar}) 
    hefty_age_data = hefty_age_data.replace({"col4":{np.nan:Dpar_mean}}) 
    hefty_age = pd.concat([hefty_age_header, hefty_age_data], axis=0)
    hefty_age.to_csv(r'{0}/{1}_HeftyAGE(Dpar).txt'.format(data_folder, sample_name), na_rep=0, index=None, sep='\t')
            ###using rmr0
    if 'rmr0' in age_df.columns:
        hefty_age_header_rmr0 = pd.DataFrame({sample_name:["Zeta:","zeta","4352.32","Ns"],
                                    "col1":["LAICPMS ratio","sig zeta","1.1","Area (cm²)"],
                                    "col2":["","","","Pcorr"],
                                    "col3":["","","","sig(Pcorr)"],
                                    "col4":["","","","rmr0"]})   
        hefty_age_data_rmr0 = pd.DataFrame({sample_name:ns,
                                     "col1":area,
                                     "col2":U_ppm,
                                     "col3":U_ppm_1sigma,
                                     "col4":rmr0}) 
        hefty_age_data_rmr0 = hefty_age_data_rmr0.replace({"col4":{np.nan:rmr0_mean}}) 
        hefty_age_rmr0 = pd.concat([hefty_age_header_rmr0, hefty_age_data_rmr0], axis=0)
        hefty_age_rmr0.to_csv(r'{0}/{1}_HeftyAGE(rmr0).txt'.format(data_folder, sample_name), na_rep=0, index=None, sep='\t')

    ################################
        #Length File:
    if mtl != '--':
        hefty_length = pd.DataFrame({"length":lengths_data['True Length'],
                    "angle":lengths_data['Angle to CAxis'],"Dpar":lengths_data['Average DPar(µmm)']})
        hefty_length = hefty_length.replace({"Dpar":{np.nan:np.mean(lengths_data['Average DPar(µmm)'])}})
        hefty_length.to_csv(r'{0}/{1}_HeftyLENGTH(Dpar).txt'.format(data_folder, sample_name), na_rep=0, index=None, sep='\t')
    
    if mtl != '--':
        hefty_length_rmr0 = pd.DataFrame({"length":lengths_data['True Length'],
                    "angle":lengths_data['Angle to CAxis'],"rmr0":rmr0D_lengths})
        hefty_length_rmr0 =  hefty_length_rmr0.replace({"rmr0":{np.nan:np.mean(rmr0D_lengths)}})
        hefty_length_rmr0.to_csv(r'{0}/{1}_HeftyLENGTH(rmr0).txt'.format(data_folder, sample_name), na_rep=0, index=None, sep='\t')

    ###############################################################################################################
    #export tables to use in RadialPlotter
    
        #ColorwUppm:
    radialplotter_U = pd.DataFrame({sample_name:Age_sga.round(2),"O":Age_sga_1sigma.round(2),"":U_ppm.round(2)})
    radialU_path = '{0}/{1}_Radial_U.csv'.format(data_folder, sample_name)
    radialplotter_U.to_csv(r'{0}'.format(radialU_path), na_rep=0, index=None)
    
        #ColorwDpar:
    radialplotter_Dpar = pd.DataFrame({sample_name:Age_sga.round(2),"O":Age_sga_1sigma.round(2),"":Dpar.round(2)})
    radialplotter_Dpar = radialplotter_Dpar.replace({"":{np.nan:round(np.mean(Dpar), 3)}})
    radialDpar_path = '{0}/{1}_Radial_Dpar.csv'.format(data_folder, sample_name)
    radialplotter_Dpar.to_csv(r'{0}'.format(radialDpar_path), na_rep=0, index=None)
    
        #Colorw rmr0 calculated from Dpar
    radialplotter_rmr0D = pd.DataFrame({sample_name:Age_sga.round(2),"O":Age_sga_1sigma.round(2),"":rmr0D})
    radialplotter_rmr0D =  radialplotter_rmr0D.replace({"":{np.nan:round(np.mean(rmr0D), 3)}})
    radialrmr0D_path = '{0}/{1}_Radial_rmr0(From_Dpar).csv'.format(data_folder, sample_name)
    radialplotter_rmr0D.to_csv(r'{0}'.format(radialrmr0D_path), na_rep=0, index=None)
       
        #ColorwCl (if cl is there)
    if np.issubdtype(ns_df['Cl wt (%)'].dtype, np.number):
        radialplotter_Cl = pd.DataFrame({sample_name:Age_sga.round(2),"O":Age_sga_1sigma.round(2),"":Cl_sem})
        radialplotter_Cl =  radialplotter_Cl.replace({"":{np.nan:round(np.mean(Cl_sem), 3)}})
        radialCl_path = '{0}/{1}_Radial_Cl.csv'.format(data_folder, sample_name)
        radialplotter_Cl.to_csv(r'{0}'.format(radialCl_path), na_rep=0, index=None)
        
        #Colorwrmr0 (if rmr0 is there)
    if np.issubdtype(ns_df['rmr0'].dtype, np.number):
        radialplotter_rmr0 = pd.DataFrame({sample_name:Age_sga.round(2),"O":Age_sga_1sigma.round(2),"":rmr0})
        radialplotter_rmr0 =  radialplotter_rmr0.replace({"":{np.nan:round(np.mean(rmr0), 3)}})
        radialrmr0_path = '{0}/{1}_Radial_rmr0.csv'.format(data_folder, sample_name)
        radialplotter_rmr0.to_csv(r'{0}'.format(radialrmr0_path), na_rep=0, index=None)
    ###############################################################################################################
    #export tables to use in QTQT (must copy and paste into QTQT)
        #age
    if 'rmr0' in age_df.columns:
        qtqt_age = pd.DataFrame({'Sample:{0}\n Ns'.format(sample_name): ns,"ICPMS_Age":Age_sga.round(2),
                              "ICPMS_Age 1Sigma":Age_sga_1sigma.round(2),"Dpar":Dpar.round(2),"Cl":Cl_sem,"rmr0":rmr0})
        qtqt_age = qtqt_age.replace({"Dpar":{np.nan:round(np.mean(Dpar), 2)}})
        qtqt_age = qtqt_age.replace({"Cl":{np.nan:round(np.mean(Cl_sem), 3)}})
        qtqt_age = qtqt_age.replace({"rmr0":{np.nan:round(np.mean(rmr0), 3)}})
        qtqt_age.to_csv(r'{0}/{1}_QTQT_age.csv'.format(data_folder, sample_name), na_rep=0, index=None)
    else:
        qtqt_age = pd.DataFrame({'Sample:{0}\n Ns'.format(sample_name): ns,"ICPMS_Age":Age_sga.round(2),
                              "ICPMS_Age 1Sigma":Age_sga_1sigma.round(2),"Dpar":Dpar.round(2),"Cl":Cl_sem,"rmr0":rmr0D})
        qtqt_age = qtqt_age.replace({"Dpar":{np.nan:round(np.mean(Dpar), 2)}})
        qtqt_age = qtqt_age.replace({"rmr0":{np.nan:round(np.mean(rmr0D), 3)}})
        qtqt_age.to_csv(r'{0}/{1}_QTQT_age.csv'.format(data_folder, sample_name), na_rep=0, index=None)

        #length
    if mtl != '--':
        qtqt_length = pd.DataFrame({'Sample:{0}\n Length'.format(sample_name): lengths_data['True Length'],"Angle to C":lengths_data['Angle to CAxis'],
                              "Dpar":lengths_data['Average DPar(µmm)'],"rmr0_Dpar":rmr0D_lengths})
        qtqt_length = qtqt_length.replace({"Dpar":{np.nan:round(np.mean(lengths_data['Average DPar(µmm)']),2)}})
        qtqt_length = qtqt_length.replace({"rmr0_Dpar":{np.nan:round(np.mean(rmr0D_lengths), 3)}})
        qtqt_length.to_csv(r'{0}/{1}_QTQT_length.csv'.format(data_folder, sample_name), na_rep=0, index=None)
    ###############################################################################################################
    #export table to use in IsoplotR
    isoplot_r = pd.DataFrame({'Sample:{0}\n Ns'.format(sample_name): ns,"A(µm2)":(area*10**8).round(0),
                              "U1(ppm)":U_ppm.round(2),"err[U1]":U_ppm_1sigma.round(2)})
    isoplot_r.to_csv(r'{0}/{1}_IsoplotR.csv'.format(data_folder, sample_name), na_rep=0, index=None)
    ###############################################################################################################

In [19]:
# map_data = 1
# if map_data != 0:
#     def map_sample(b):
#         display(Javascript('IPython.notebook.execute_cell()'))
#         fig = plt.figure(constrained_layout=True, figsize=(15,10),)
#         spec = gs.GridSpec(ncols=1, nrows=1, figure=fig)
#
#         ax = fig.add_subplot(spec[0, 0])
#         map = Basemap(llcrnrlon=append_pooled_df['longitude'].min()-0.5,llcrnrlat=append_pooled_df['latitude'].min()-0.5,
#                       urcrnrlon=append_pooled_df['longitude'].max()+0.5,urcrnrlat=append_pooled_df['latitude'].max()+0.5,
#                       ax=ax, resolution='l',epsg=4326)
#
#         map.arcgisimage(service='ESRI_Imagery_World_2D', xpixels = 400, verbose= True)
#         map.drawcountries(color='k')
#         map.scatter(append_pooled_df['longitude'], append_pooled_df['latitude'], latlon=True,
#               color='r', s=10,alpha=0.8, label="Other Samples")
#         map.scatter(longitude, latitude, latlon=True,
#               color='#e0c200', s=20, alpha=0.8, label="This Sample")
#         ax.annotate(round(Central_age,1),(longitude+0.01,latitude+0.01),
#                        fontsize=10,
#                        color='#e0c200')
#         for i, txt in enumerate(append_pooled_df['cage']):
#             ax.annotate(append_pooled_df['cage'][i],
#                         (append_pooled_df['longitude'][i]+0.01,append_pooled_df['latitude'][i]+0.01),
#                        fontsize=10,
#                        color='r')
#         ax.legend(loc='upper right')
#
#         #ax.text(longitude+0.02, latitude+0.02, sample_name,
#         #      color='#e0c200',alpha=0.8, fontsize=16, weight='bold')
#
#     map_loc = widgets.Button(description = "Map All Sample Locations",button_style='success')
#     map_loc.on_click(map_sample)
#
#
# display(map_loc)
# print("Might Take a Minute")

In [20]:
if counts_file_w.value != {} or counts_paths != []:
    print('Done', date.today())
    display(ns_df.style.hide_index())

In [21]:
if counts_file_w.value != {} or counts_paths != []:
    print('7) OPTIONAL: PATH TO RADIAL PLOTTER .JAR FILE')
    radial_plotter_w = widgets.Text(
        value='RadialPlotter.jar',
        placeholder='/Users/Desktop/RadialPlotter.jar',
        description='Radial .jar:',
        layout=Layout(width='90%', height='100%'),
        disabled=False)

    def clear_rfolder(b):
        radial_plotter_w.value = ""
    clear_rbutton = widgets.Button(description = "Clear Path", button_style='danger')
    clear_rbutton.on_click(clear_rfolder)


    display(widgets.VBox([widgets.HBox([radial_plotter_w]),widgets.HBox([clear_rbutton])]))

In [22]:
if counts_file_w.value != {} or counts_paths != []:
    print('7.1) Open Radial Plotter to save certain settings, separate from those below\n    (colors, transformation type, Z label)\n')

    def radial_settings(b):
        subprocess.call(['java', '-jar', radial_plotter_w.value,
                         'in='])

    rsettings_button = widgets.Button(description = "Radial Plotter Settings", button_style='primary',
                                      layout=Layout(width='47%'))
    rsettings_button.on_click(radial_settings)


    display(widgets.HBox([rsettings_button]))
    print('--Change settings, then close the window and continue--')

In [23]:
if counts_file_w.value != {} or counts_paths != []:
    print('7.2) Plot based on the variable for the color\n')
    print('-->only # of mixture models is variable here,\n to change colors, transformation or Z label, use\n "Radial Plotter Settings button above"<--')


    #middle mixture models
    mixture_w = widgets.Select(
        options=[('0 Mix. Model', 0), ('1 Mix. Model', 1),( '2 Mix. Model', 2),
                 ('3 Mix .Model', 3), ('4 Mix. Model', 4),
                ('5 Mix. Model', 5)],
        layout=Layout(flex='1 1 auto', width='auto'),
        value=0,
        rows=6,
        #description='OS:',
        disabled=False
    )


    #right side image
    ex_img = open('example.jpg', 'rb').read()
    ex_im_w = widgets.Image(value=ex_img, format='jpg', 
                            layout=Layout(flex='2 1 auto', width='auto'),
                            width=200)

    #left side
    #Dpar
    def color_dpar(b):
        radial_folder = '{0}/Radial_Plots'.format(up_onefolder)
        if not os.path.exists(radial_folder): os.makedirs(radial_folder)

        subprocess.call(['java', '-jar', radial_plotter_w.value,
                         'numpeaks={}'.format(mixture_w.value),
                         'in={}'.format(radialDpar_path),
                        'out={0}/{1}_Dpar{2}mix.pdf'.format(radial_folder,sample_name,mixture_w.value)])
        with out_dpar:
            print('File saved to:',radial_folder)



    #U ppm
    def color_U(b):
        radial_folder = '{0}/Radial_Plots'.format(up_onefolder)
        if not os.path.exists(radial_folder): os.makedirs(radial_folder)

        subprocess.call(['java', '-jar', radial_plotter_w.value,
                         'numpeaks={}'.format(mixture_w.value),
                         'in={}'.format(radialU_path),
                        'out={0}/{1}_U{2}mix.pdf'.format(radial_folder,sample_name,mixture_w.value)])
        with out_U:
            print('File saved to:',radial_folder)



    if 'rmr0' in age_df.columns:
        #Cl wt%
        def color_Cl(b):
            radial_folder = '{0}/Radial_Plots'.format(up_onefolder)
            if not os.path.exists(radial_folder): os.makedirs(radial_folder)

            subprocess.call(['java', '-jar', radial_plotter_w.value,
                             'numpeaks={}'.format(mixture_w.value),
                             'in={}'.format(radialCl_path),
                            'out={0}/{1}_Cl{2}mix.pdf'.format(radial_folder,sample_name,mixture_w.value)])
            with out_Cl:
                print('File saved to:',radial_folder)




        #rmr0
        def color_rmr0(b):
            radial_folder = '{0}/Radial_Plots'.format(up_onefolder)
            if not os.path.exists(radial_folder): os.makedirs(radial_folder)

            subprocess.call(['java', '-jar', radial_plotter_w.value,
                             'numpeaks={}'.format(mixture_w.value),
                             'in={}'.format(radialrmr0_path),
                            'out={0}/{1}_rmr0{2}mix.pdf'.format(radial_folder,sample_name,mixture_w.value)])
            with out_rmr0:
                print('File saved to:',radial_folder)

        color_Cl_button = widgets.Button(description = "Plot with wt% Cl", button_style='',
                                         layout=Layout(flex='2 1 auto', width='auto'))

        color_rmr0_button = widgets.Button(description = "Plot with rmr0", button_style='',
                                         layout=Layout(flex='2 1 auto', width='auto'))
        
        color_dpar_button = widgets.Button(description = "Plot with Dpar", button_style='',
                                         layout=Layout(flex='2 1 auto', width='auto'))

        color_U_button = widgets.Button(description = "Plot with [U]", button_style='',
                                          layout=Layout(flex='2 1 auto', width='auto'))

        color_Cl_button.on_click(color_Cl)
        out_Cl = Output()

        color_rmr0_button.on_click(color_rmr0)
        out_rmr0 = Output()
        
        color_dpar_button.on_click(color_dpar)
        out_dpar = Output()

        color_U_button.on_click(color_U)
        out_U = Output()

        items_2 = [Label('Select Composition'),
               color_dpar_button, color_U_button, color_Cl_button, color_rmr0_button]
        items_4 = [out_dpar, out_U, out_Cl, out_rmr0]

    else:
        color_dpar_button = widgets.Button(description = "Plot with Dpar", button_style='',
                                         layout=Layout(flex='2 1 auto', width='auto'))

        color_U_button = widgets.Button(description = "Plot with [U]", button_style='',
                                          layout=Layout(flex='2 1 auto', width='auto'))
        
        color_dpar_button.on_click(color_dpar)
        out_dpar = Output()

        color_U_button.on_click(color_U)
        out_U = Output()
    
        
        items_2 = [Label('Select Composition'),
                   color_dpar_button, color_U_button]
        items_4 = [out_dpar, out_U]







    items_1 = [Label('# of Mixture Models'),mixture_w]
    items_3 = [Label('Ex. plot w/ 2 mixture models, color w/ Dpar'),ex_im_w]

    box_layout = Layout(display='flex',
                        flex_flow='column',
                        align_items='stretch',
                        width='30%')
    box_layout2 = Layout(display='flex',
                        flex_flow='column',
                        align_items='stretch',
                        width='15%')

    box_1 = Box(children=items_1, layout=box_layout2)
    box_2 = Box(children=items_2, layout=box_layout)
    box_3 = Box(children=items_3, layout=box_layout)
    box_4 = Box(children=items_4, layout=box_layout)

    display(VBox([(HBox([box_1, box_2, box_3])),(HBox([box_4]))]))

In [24]:
if counts_file_w.value != {} or counts_paths != []:
    if mtl != '--':
        def histograms(b):
            mtl_folder = '{0}/Histograms'.format(up_onefolder)
            if not os.path.exists(mtl_folder): os.makedirs(mtl_folder)

            bins_c = color_Bins_w.value
            stroke_c = color_Stroke_w.value
            KDE_c = color_KDE_w.value
            bins = bins_w.value
            histtype = bins_type_w.value
            #first figure; large, easy to read, with all data. Good for appendix
            fig1 = plt.figure(constrained_layout=False, figsize=(3,3))
            spec = gs.GridSpec(nrows=1,ncols=1, figure=fig1)
            ax = plt.subplot(spec[0,0])
            ax.hist(true_length, bins=bins, range=(2,17),  histtype=histtype, 
                    facecolor=bins_c,edgecolor=stroke_c, linewidth=1.5, alpha=0.8)
                #Creating another Y axis
            second_ax = ax.twinx()
                #Plotting kde without hist on the second Y axis
            if KDE_w.value ==1:
                sns.kdeplot(true_length, ax=second_ax, color=KDE_c, linewidth=3, alpha=0.8)
            ax.axvline(mtl, linewidth=1.5, linestyle='--', color='k', alpha=0.7, label='MTL')
            ax.text(0.015, 0.985, 'MTL = {0:.1f}±{1:.1f} µm\nSD = {2:.1f} µm\nN = {3}' .format(mtl, mtl_var, mtl_sd, l_no),
                         horizontalalignment='left',verticalalignment='top', transform=ax.transAxes, fontsize=10)
            ax.set_ylabel('Count', fontsize=11)
            second_ax.set_ylabel('Relative Frequency', fontsize=11)
            ax.set_xlabel('Track Length (µm)', fontsize=11)
            ax.tick_params(axis="x", labelsize=10)
            ax.tick_params(axis="y", labelsize=10)
            ax.autoscale(enable=True, axis='x', tight=True)
            ax.set_xticks(ticks=[2,4,6,8,10,12,14,16], minor=False)
            ax.set_yticks(ticks=[0,10,20,30,40,50], minor=False)
            ax.set_ylim([0,50])
            second_ax.set_ylim([0,0.5])
            ax.set_xlim([2,17])
            ax.set_title("{0}".format(sample_name), fontdict={'fontsize': "11"}, loc='left')


                #2nd histogram plot, smaller and good for maps/used in other plots
            fig2 = plt.figure(constrained_layout=False, figsize=(1.3,1.3))
            spec = gs.GridSpec(nrows=1,ncols=1, figure=fig2)
            ax2 = plt.subplot(spec[0,0])
            ax2.hist(true_length, bins=bins, range=(2,17), histtype=histtype,
                             facecolor=bins_c,edgecolor=stroke_c, linewidth=0.8, alpha=0.8)
                #Creating another Y axis
            second_ax2 = ax2.twinx()
                #Plotting kde without hist on the second Y axis
            if KDE_w.value ==1:
                sns.kdeplot(true_length, ax=second_ax2, color=KDE_c, linewidth= 1.5, alpha=0.8)
            sns.despine(fig=None, ax=second_ax2, top=True, right=True, left=False, bottom=False, offset=None, trim=False)
                #Removing Y ticks from the second axis
            second_ax2.set_yticks([])
            second_ax2.get_yaxis().set_visible(False)
            ax2.text(0.015, 1.0, '{0}\nMTL: {1:.1f}±{2:.1f}\n' .format(sample_name, mtl, mtl_var),
                         horizontalalignment='left',verticalalignment='top', transform=ax2.transAxes, fontsize=8)
            ax2.set_xticks(ticks=[4,8,12,16], minor=False)
            ax2.tick_params(axis="x", labelsize=8)
            ax2.tick_params(axis="y", labelsize=8)
            ax2.spines['top'].set_visible(False)
            ax2.spines['right'].set_visible(False)
            ax2.set_ylim([0,50])
            second_ax2.set_ylim([0,0.5])
            ax2.set_xlim([2,17])

                #3rd histogram plot, much much smaller for use in tight maps etc
            fig3 = plt.figure(constrained_layout=False, figsize=(0.6,0.6))
            spec = gs.GridSpec(nrows=1,ncols=1, figure=fig3)
            ax3 = plt.subplot(spec[0,0])
            ax3.hist(true_length, bins=bins, range=(2,17), color='grey', alpha=0.6) 
                #Creating another Y axis
            second_ax3 = ax3.twinx()
                #Plotting kde without hist on the second Y axis
            sns.kdeplot(true_length, ax=second_ax3, color="k", linewidth=1, alpha=1)
            sns.despine(fig=None, ax=second_ax3, top=True, right=True, left=True, bottom=True, offset=None, trim=False)
                #Removing Y ticks from the second axis
            second_ax3.set_yticks([])
            second_ax3.get_yaxis().set_visible(False)
            ax3.text(0.5, -0.2, '{0:.1f}±{1:.1f}' .format(mtl,mtl_sd),
                         horizontalalignment='center',verticalalignment='center', transform=ax3.transAxes, fontsize=9)
            ax3.get_xaxis().set_ticks([])
            ax3.get_yaxis().set_ticks([])
            ax3.set_ylim([0,40])
            second_ax3.set_ylim([0,0.4])
            ax3.set_xlim([2,17])

            with out_mtl:
                #plt.show(fig1)
                print('File saved to:',mtl_folder)

            file_path1 = "{0}/{1}_l.pdf".format(mtl_folder,sample_name)
            fig1.savefig(file_path1, dpi=300, bbox_inches='tight')
            plt.close()

            file_path2 = "{0}/{1}_m.pdf".format(mtl_folder,sample_name)
            fig2.savefig(file_path2, dpi=300, bbox_inches='tight')
            plt.close() 


            file_path3  = "{0}/{1}_s.pdf".format(mtl_folder,sample_name)
            fig3.savefig(file_path3, dpi=300, bbox_inches='tight')
            plt.close() 


In [25]:
if counts_file_w.value != {} or counts_paths != []:    
    if mtl != '--':
        print("8) GENERATE TRACK LENGTH HISTOGRAMS")

        histograms_button = widgets.Button(description = "Generate TL Histograms",
                                        button_style='primary', layout=Layout(width='47%'))
        histograms_button.on_click(histograms)

            #example image
        mtl_img = open('example_mtl.jpg', 'rb').read()
        mtl_im_w = widgets.Image(value=mtl_img, format='jpg', 
                                    layout=Layout(flex='1 1 auto', width='auto'),
                                    width=200)

        color_KDE_w = widgets.ColorPicker(
            concise=False,
            description='KDE Color',
            layout=Layout(flex='1 1 auto', width='auto'),
            value='#6495ED',
            disabled=False
        )

        KDE_w = widgets.Select(
            options=[("On",1), ("Off",0)],
            value=1,
            layout=Layout(flex='1 1 auto', width='auto'),
            rows=2,
            description='KDE on/off',
            disabled=False)


        color_Bins_w = widgets.ColorPicker(
            concise=False,
            description='Bins Color',
            layout=Layout(flex='1 1 auto', width='auto'),
            value='#CCCCCC',
            disabled=False
        )

        bins_type_w = widgets.Select(
            options=['bar', 'barstacked', 'step', 'stepfilled'],
            value='stepfilled',
            layout=Layout(flex='1 1 auto', width='auto'),
            rows=4,
            description='Bar Type',
            disabled=False)

        bins_w = widgets.IntText(
            value=15,
            description='No. of Bins:',
            layout=Layout(flex='1 1 auto', width='auto'),
            step=1,
            disabled=False,)

        color_Stroke_w = widgets.ColorPicker(
            concise=False,
            description='Stroke Color',
            layout=Layout(flex='1 1 auto', width='auto'),
            value='#000000',
            disabled=False
        )

        out_mtl = Output()


        mtlitems_1 = [Label('Example Histogram w Default Colors'),mtl_im_w]
        mtlitems_2 = [color_KDE_w,KDE_w,color_Bins_w,bins_type_w,bins_w,color_Stroke_w]
        mtlitems_3 = [histograms_button, out_mtl]


        mtl_layout = Layout(display='flex',
                                flex_flow='column',
                                align_items='stretch',
                                width='30%')

        mtl_layout2 = Layout(display='flex',
                                flex_flow='column',
                                align_items='stretch',
                                width='100%')

        mtlbox_1 = Box(children=mtlitems_1, layout=mtl_layout)
        mtlbox_2 = Box(children=mtlitems_2, layout=mtl_layout)
        mtlbox_3 = Box(children=mtlitems_3, layout=mtl_layout2)


        display(VBox([(HBox([mtlbox_1, mtlbox_2])),(HBox([mtlbox_3]))]))